# Cleaning of Job Data

**Cleaning Part 1:**

First, the Location, Company Name, Job Titles, and Seniority Level are cleaned. Those columns will then be used to detect duplicates.


**Removal of Duplicates:**

The dataframe will be scanned for duplicated rows in multiple steps:
1. Drop rows that are duplicates based on all columns.
2. Drop rows that show duplicated values in a selection of columns.
3. Compare written job descriptions, flagging and dropping listings that show substantial similarity.


**Cleaning Part 2:**

After having removed duplicated listings, the following cleaning steps will be performed:
- Job Titles: titles will be categorized into 10 groups with the help of ChatGPT.
- Number of Days Online: The number of days a job ad has been online will be grouped.
- Tech Stack: Technologies will be defined and extracted if mentioned in job description.
- Geodata: Based on the cleaned location column, latitude and longitude of the individual locations will be added.
- Long format: the final dataframe will additionally be stored in a long format allowing for filtering of mentioned tech stack and job types in a streamlit dashboard.

In [1]:
# Import libraries
import Levenshtein
import numpy as np
import pandas as pd
import pickle

from geopy.geocoders import Nominatim

In [2]:
# Read file
file_name = "job_data_raw_anonymized.csv"
df = pd.read_csv(file_name)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1625 entries, 0 to 1624
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   title                        1625 non-null   object 
 1   company                      1625 non-null   object 
 2   location                     1625 non-null   object 
 3   salary                       22 non-null     object 
 4   seniority_level              796 non-null    object 
 5   date_posted                  1147 non-null   object 
 6   date_accessed                1625 non-null   object 
 7   days_online                  1147 non-null   float64
 8   number_applicants            796 non-null    float64
 9   job_type_Vollzeit            1625 non-null   int64  
 10  job_type_Teilzeit            1625 non-null   int64  
 11  job_type_Festanstellung      1625 non-null   int64  
 12  job_type_BefristeterVertrag  1625 non-null   int64  
 13  job_type_Homeoffic

## Location

In [3]:
df["location"].value_counts()

location
Berlin                       241
München                      147
Deutschland                   91
Hamburg                       76
 Berlin                       72
                            ... 
 Bad Homburg vor der Höhe      1
 Straße                        1
Bremerhaven                    1
 Wuppertal                     1
Münchberg                      1
Name: count, Length: 302, dtype: int64

In [4]:
# clean white space
df["location"] = df["location"].str.strip()
df["location"].value_counts()

location
Berlin               324
München              200
Hamburg              110
Deutschland           96
Frankfurt am Main     87
                    ... 
Penzberg               1
Büren                  1
Gießen                 1
Buch                   1
Münchberg              1
Name: count, Length: 236, dtype: int64

In [5]:
np.sort(df["location"].unique())

array(['Aachen',
       'Aachen, Bonn, Dortmund, Düsseldorf, Essen, Frankfurt, Hannover, Köln, Münster, Paderborn, Siegen',
       'Aalen', 'Ahrensburg', 'Arnstorf', 'Aschheim', 'Attenkirchen',
       'Augsburg',
       'Augsburg, Karlsruhe, Koblenz, München, Nürnberg, Saarbrücken, Stuttgart, Ulm, Walldorf',
       'Bad Berleburg', 'Bad Ems', 'Bad Gandersheim', 'Bad Homburg',
       'Bad Homburg vor der Höhe', 'Bad Nauheim', 'Bad Salzuflen',
       'Baden-Württemberg', 'Bargteheide', 'Bayern', 'Bayreuth',
       'Bensheim', 'Bergisch Gladbach', 'Berlin', 'Berlin-Kreuzberg',
       'Biberach', 'Biberach an der Riß', 'Bielefeld', 'Bochum', 'Bonn',
       'Braunschweig', 'Bremen', 'Bremen, Oldenburg', 'Bremerhaven',
       'Buch', 'Burgdorf', 'Burghausen', 'Böblingen', 'Büren',
       'Clausthal-Zellerfeld', 'Coburg', 'Coesfeld', 'Coesfeld-Lette',
       'Damme', 'Darmstadt', 'Detmold', 'Deutschland', 'Dorsten',
       'Dortmund', 'Dresden', 'Duisburg', 'Düsseldorf',
       'Düsseldorf, P

In [6]:
# show listings with more than one location
df.loc[df["location"].str.contains(","), "location"]

1300                                    Nürnberg, München
1302                                    Bremen, Oldenburg
1310                         Stuttgart, Frankfurt am Main
1337                               Osnabrück, Saarbrücken
1370                           Frankfurt am Main, München
1372                                Düsseldorf, Paderborn
1382                                        St. Leon-Rot,
1388                                        Hamburg, Köln
1398                                      Mannheim, Fürth
1460                                   Stuttgart, Hamburg
1483                   Stuttgart, Esslingen, Sindelfingen
1505    Hamburg, München, Ulm, Donauwörth, Friedrichsh...
1527                                     Köln, Düsseldorf
1532    Fischach, Augsburg, Freising, München, Wachau,...
1557                                    München, Tübingen
1578    Augsburg, Karlsruhe, Koblenz, München, Nürnber...
1579    Aachen, Bonn, Dortmund, Düsseldorf, Essen, Fra...
1585          

In [7]:
# For simplicity, use first location mentioned
# Create regular expression removing all characters after a comma
my_regex = r"(,).*$"

# Test regex
df.loc[df["location"].str.contains(","), "location"].str.replace(my_regex, "", regex=True)

1300             Nürnberg
1302               Bremen
1310            Stuttgart
1337            Osnabrück
1370    Frankfurt am Main
1372           Düsseldorf
1382         St. Leon-Rot
1388              Hamburg
1398             Mannheim
1460            Stuttgart
1483            Stuttgart
1505              Hamburg
1527                 Köln
1532             Fischach
1557              München
1578             Augsburg
1579               Aachen
1585                 Jena
Name: location, dtype: object

In [8]:
# Apply regex
my_regex = r"(,).*$"
df["location_clean"] = df["location"].str.replace(my_regex, "", regex=True)
np.sort(df["location_clean"].unique())

array(['Aachen', 'Aalen', 'Ahrensburg', 'Arnstorf', 'Aschheim',
       'Attenkirchen', 'Augsburg', 'Bad Berleburg', 'Bad Ems',
       'Bad Gandersheim', 'Bad Homburg', 'Bad Homburg vor der Höhe',
       'Bad Nauheim', 'Bad Salzuflen', 'Baden-Württemberg', 'Bargteheide',
       'Bayern', 'Bayreuth', 'Bensheim', 'Bergisch Gladbach', 'Berlin',
       'Berlin-Kreuzberg', 'Biberach', 'Biberach an der Riß', 'Bielefeld',
       'Bochum', 'Bonn', 'Braunschweig', 'Bremen', 'Bremerhaven', 'Buch',
       'Burgdorf', 'Burghausen', 'Böblingen', 'Büren',
       'Clausthal-Zellerfeld', 'Coburg', 'Coesfeld', 'Coesfeld-Lette',
       'Damme', 'Darmstadt', 'Detmold', 'Deutschland', 'Dorsten',
       'Dortmund', 'Dresden', 'Duisburg', 'Düsseldorf', 'Ebermannstadt',
       'Einbeck', 'Erlangen', 'Esch', 'Eschborn', 'Essen', 'Ettlingen',
       'Fellbach', 'Fischach', 'Frankfurt', 'Frankfurt am Main',
       'Freiburg', 'Freiburg im Breisgau', 'Fürth', 'Garching',
       'Gerlingen', 'Gießen', 'Gladbeck', 

In [9]:
# Some locations need extra attention
location_dict = {
    "Biberach": "Biberach an der Riß",
    "Hannover-Braunschweig-Göttingen-Wolfsburg Region": "Hannover",
    "Ludwigshafen am Rhein": "Ludwigshafen",
    "Berlin-Kreuzberg": "Berlin",
    "Neukölln": "Berlin",
    "Munich": "München",
    "Homeoffice": "Deutschland",
}

for old_location, new_location in location_dict.items():
    df.loc[df["location_clean"] == old_location, "location_clean"] = new_location

np.sort(df["location_clean"].unique())

array(['Aachen', 'Aalen', 'Ahrensburg', 'Arnstorf', 'Aschheim',
       'Attenkirchen', 'Augsburg', 'Bad Berleburg', 'Bad Ems',
       'Bad Gandersheim', 'Bad Homburg', 'Bad Homburg vor der Höhe',
       'Bad Nauheim', 'Bad Salzuflen', 'Baden-Württemberg', 'Bargteheide',
       'Bayern', 'Bayreuth', 'Bensheim', 'Bergisch Gladbach', 'Berlin',
       'Biberach an der Riß', 'Bielefeld', 'Bochum', 'Bonn',
       'Braunschweig', 'Bremen', 'Bremerhaven', 'Buch', 'Burgdorf',
       'Burghausen', 'Böblingen', 'Büren', 'Clausthal-Zellerfeld',
       'Coburg', 'Coesfeld', 'Coesfeld-Lette', 'Damme', 'Darmstadt',
       'Detmold', 'Deutschland', 'Dorsten', 'Dortmund', 'Dresden',
       'Duisburg', 'Düsseldorf', 'Ebermannstadt', 'Einbeck', 'Erlangen',
       'Esch', 'Eschborn', 'Essen', 'Ettlingen', 'Fellbach', 'Fischach',
       'Frankfurt', 'Frankfurt am Main', 'Freiburg',
       'Freiburg im Breisgau', 'Fürth', 'Garching', 'Gerlingen', 'Gießen',
       'Gladbeck', 'Gotha', 'Grafing bei München', 

In [10]:
df["location_clean"].value_counts()

location_clean
Berlin               326
München              212
Hamburg              112
Deutschland          109
Frankfurt am Main     88
                    ... 
Penzberg               1
Büren                  1
Gießen                 1
Urbar                  1
Münchberg              1
Name: count, Length: 213, dtype: int64

## Company Names

In [11]:
# Explore unique company names
np.sort(df["company"].unique()).tolist()

['1&1 Mail & Media Applications SE',
 '1KOMMA5°',
 '1Komma5° GmbH',
 '1komma6 - Multimediale Dienstleistungen GmbH',
 '1x2 Technologies GmbH',
 '4.screen GmbH',
 '4screen',
 'ABB',
 'ABOUT YOU',
 'ABOUT YOU AG & Co. KG',
 'ACCURE Battery Intelligence',
 'ACONEXT',
 'ACONEXT Stuttgart GmbH',
 'ADAC',
 'ADAC Allgemeiner Deutscher Automobil Club',
 'ADEAL Systems GmbH',
 'ADG Apotheken-Dienstleistungsgesellschaft mbH',
 'ADM',
 'AI Futures',
 'AI.Associates GmbH',
 'AITAD GmbH',
 'AL Solutions',
 'ALACRiS Theranostics GmbH',
 'ALDI Einkauf SE & Co. oHG',
 'ALDI SÜD',
 'ALTEN Consulting Services GmbH',
 'ALTEN Germany',
 'AMAI GmbH',
 'AMD',
 'AOK Bayern - Die Gesundheitskasse',
 'ATH Altonaer-Technologie-Holding GmbH',
 'ATLAS ELEKTRONIK GmbH',
 'ATLAS MARIDAN ApS',
 'ATOSS Software AG',
 'AUTO1 Group',
 'AVAT Automation GmbH',
 'AWS EMEA SARL (Germany Branch)',
 'AXULUS',
 'AbbVie',
 'AbbVie Deutschland GmbH & Co. KG',
 'Accenture',
 'Aerzener Maschinenfabrik',
 'Aignostics',
 'Ailio Gmb

In order to consolidate company names, the column will undergo several steps of cleaning.

In [12]:
# First, remove common legal terms describing the type of company
df["company_clean_ish"] = df["company"]

replace_strings = [
    "GmbH",
    "GMBH",
    "AG",
    "a.G.",
    "a. G.",
    "KGaA",
    "Co KG",
    "Co. KG",
    "CO. KG",
    "Co Mack KG",
    "Deutschland",
    "Gruppe",
    "Group",
    "Holding",
    "B.V.",
    "SE",
    "DACH"
]

for s in replace_strings:
    string = f" {s}" # add blank in order to not replace e.g. AG in the middle of a sentence.
    df["company_clean_ish"] = df["company_clean_ish"].str.replace(string, "")

df.loc[df["company"] != df["company_clean_ish"], ["company", "company_clean_ish"]].head()

,company,company_clean_ish
30,Enzo Tech Group,Enzo Tech
34,MAM Gruppe,MAM
46,IOTIS GmbH,IOTIS
47,elunic AG,elunic
48,Source Group International,Source International


In [13]:
# Remove all non-word charachters; keep blanks
df["company_clean_ish"] = df["company_clean_ish"].str.replace("[^a-zA-Z0-9\s]", " ", regex=True)

# show datapoints that have changed:
df.loc[df["company"] != df["company_clean_ish"], ["company", "company_clean_ish"]].head()

,company,company_clean_ish
2,Global Market Solutions - Germany,Global Market Solutions Germany
4,vent.io,vent io
7,HUK-COBURG,HUK COBURG
13,CONNECTMETA.AI,CONNECTMETA AI
18,SR2 | Socially Responsible Recruitment | Certi...,SR2 Socially Responsible Recruitment Certi...


In [14]:
# remove leading/trailing blanks
df["company_clean_ish"] = df["company_clean_ish"].str.strip()

In [15]:
# Transform company names to upper 
df["company_clean_ish"] = df["company_clean_ish"].str.upper()

In [16]:
# Check unique list of company names to find spellings of company names that can be combined.
np.sort(df["company_clean_ish"].unique()).tolist()

['1 1 MAIL   MEDIA APPLICATIONS',
 '1KOMMA5',
 '1KOMMA6   MULTIMEDIALE DIENSTLEISTUNGEN',
 '1X2 TECHNOLOGIES',
 '4 SCREEN',
 '4SCREEN',
 'ABAT NORTH AMERICA',
 'ABB',
 'ABBVIE',
 'ABOUT YOU',
 'ACCANTEC GROUP',
 'ACCENTURE',
 'ACCURE BATTERY INTELLIGENCE',
 'ACONEXT',
 'ACONEXT STUTTGART',
 'ADAC',
 'ADAC ALLGEMEINER DEUTSCHER AUTOMOBIL CLUB',
 'ADEAL SYSTEMS',
 'ADESSO',
 'ADG APOTHEKEN DIENSTLEISTUNGSGESELLSCHAFT MBH',
 'ADM',
 'AERZENER MASCHINENFABRIK',
 'AI ASSOCIATES',
 'AI FUTURES',
 'AIGNOSTICS',
 'AILIO',
 'AINOVI',
 'AITAD',
 'AKKODIS',
 'AKLASS SOLUTIONS',
 'AL SOLUTIONS',
 'ALACRIS THERANOSTICS',
 'ALBATROSS AI',
 'ALCEDIS   A HUMA COMPANY',
 'ALDI EINKAUF   CO  OHG',
 'ALDI S D',
 'ALEXANDER THAMM',
 'ALFRED WEGENER INSTITUT F R POLAR  UND',
 'ALGEA CARE',
 'ALITHEA BIO',
 'ALIXPARTNERS',
 'ALLGEIER PUBLIC',
 'ALLIANZ',
 'ALLIANZ DIGITAL HEALTH',
 'ALLIANZ VERSICHERUNGS AG',
 'ALLOS HOF MANUFAKTUR',
 'ALMEDIA',
 'ALTEN CONSULTING SERVICES',
 'ALTEN GERMANY',
 'AMAI',
 'AMA

In [17]:
# Check some names
df.loc[df["company"].str.upper().str.contains("ZERTIFICON"), ["company", "location", "title", "source_id"]]

,company,location,title,source_id
398,Zertificon Solutions GmbH,Neukölln,Data Analyst (m/f/d) in IT security company,1
1152,Zertificon,Berlin,Data Analyst (m/f/d) in IT security company,2


In [18]:
# create a dictionary of (parts of) company names that occure, and the name we want to replace it with.

company_dict = {
'4SCREEN': '4 SCREEN',
'ACONEXT': 'ACONEXT', #'ACONEXT STUTTGART'
'ADAC': 'ADAC', #'ADAC ALLGEMEINER DEUTSCHER AUTOMOBIL CLUB',
'AXEL SPRINGER': 'AXEL SPRINGER', #'AXEL SPRINGER NATIONAL MEDIA  TECH',
'BECHTLE BREMEN': 'BECHTLE ITSYSTEMHAUS BREMEN',
'BOOZ ALLEN': 'BOOZ ALLEN',
'CHECK24': 'CHECK24', # 'CHECK24 VERGLEICHSPORTAL',
'CHERRY VENTURES': 'CHERRY VENTURES', # 'CHERRY VENTURES MANAGEMENT'
'CHOCO': 'CHOCO COMMUNICATIONS',
'ENPAL': 'ENPAL', #'ENPAL DRAGON',
'HOPMANN': 'HOPMANN MARKETING ANALYTICS', # 'HOPMANN   MARKETING ANALYTICS INH  J RG HOPMANN',
'HUK COBURG': 'HUK COBURG', #'HUK COBURG VERSICHERUNGSGRUPPE',
'LEIBNIZ INSTITUT F R ANALYTISCHE WISSENSCHAFTEN': 'LEIBNIZ INSTITUT F R ANALYTISCHE WISSENSCHAFTEN',
'LEIBNIZ INSTITUT F R ASTROPHYSIK': 'LEIBNIZ INSTITUT F R ASTROPHYSIK', # 'LEIBNIZ INSTITUT F R ASTROPHYSIK POTSDAM',
'LEIBNIZ INSTITUT F R NATURSTOFF': 'LEIBNIZ INSTITUT F R NATURSTOFF FORSCHUNG UND INFEKTIONSBIOLOGIE E V',
'LOTUM': 'LOTUM', # 'LOTUM MEDIA'
'LUNIT': 'LUNIT', #'LUNIT CANCER SCREENING'
'MVGM':  'MVGM', # 'MVGM INTERNATIONAL'
'NETTREK': 'NETTREK', #'NETTREK UND'
'CLOPPENBURG': 'PEEK CLOPPENBURG', # 'PEEK   CLOPPENBURG     D SSELDORF'
'PREDIUM': 'PREDIUM TECHNOLOGY',
'PROCESS': 'PROCESSAND',
'T V S D': 'T V S D', # TÜV SÜD; 'T V S D DIGITAL SERVICE'
'VALUEWORKS': 'VALUEWORKS AI',
'ZERTIFICON': 'ZERTIFICON', # 'ZERTIFICON SOLUTIONS'
}

for contains_string, replace_string in company_dict.items():
    df.loc[df["company_clean_ish"].str.upper().str.contains(contains_string), ["company_clean_ish"]] = replace_string


In [19]:
# Check unique list again
np.sort(df["company_clean_ish"].unique()).tolist()

['1 1 MAIL   MEDIA APPLICATIONS',
 '1KOMMA5',
 '1KOMMA6   MULTIMEDIALE DIENSTLEISTUNGEN',
 '1X2 TECHNOLOGIES',
 '4 SCREEN',
 'ABAT NORTH AMERICA',
 'ABB',
 'ABBVIE',
 'ABOUT YOU',
 'ACCANTEC GROUP',
 'ACCENTURE',
 'ACCURE BATTERY INTELLIGENCE',
 'ACONEXT',
 'ADAC',
 'ADEAL SYSTEMS',
 'ADESSO',
 'ADG APOTHEKEN DIENSTLEISTUNGSGESELLSCHAFT MBH',
 'ADM',
 'AERZENER MASCHINENFABRIK',
 'AI ASSOCIATES',
 'AI FUTURES',
 'AIGNOSTICS',
 'AILIO',
 'AINOVI',
 'AITAD',
 'AKKODIS',
 'AKLASS SOLUTIONS',
 'AL SOLUTIONS',
 'ALACRIS THERANOSTICS',
 'ALBATROSS AI',
 'ALCEDIS   A HUMA COMPANY',
 'ALDI EINKAUF   CO  OHG',
 'ALDI S D',
 'ALEXANDER THAMM',
 'ALFRED WEGENER INSTITUT F R POLAR  UND',
 'ALGEA CARE',
 'ALITHEA BIO',
 'ALIXPARTNERS',
 'ALLGEIER PUBLIC',
 'ALLIANZ',
 'ALLIANZ DIGITAL HEALTH',
 'ALLIANZ VERSICHERUNGS AG',
 'ALLOS HOF MANUFAKTUR',
 'ALMEDIA',
 'ALTEN CONSULTING SERVICES',
 'ALTEN GERMANY',
 'AMAI',
 'AMAZON',
 'AMAZON DEVELOPMENT CENTER GERMANY',
 'AMAZON DEVELOPMENT CENTER GERMANY 

### Consolidate company names

In [20]:
# Group data by cleaned(-ish) company names, save first occurance of value in the original company name column.
company_cleaning_df = df[["company", "company_clean_ish"]].groupby("company_clean_ish", as_index=False).agg({"company": "first"})
company_cleaning_series = company_cleaning_df["company"]
company_cleaning_series.index = company_cleaning_df["company_clean_ish"]
company_cleaning_series

company_clean_ish
1 1 MAIL   MEDIA APPLICATIONS                          1&1 Mail & Media Applications SE
1KOMMA5                                                                        1KOMMA5°
1KOMMA6   MULTIMEDIALE DIENSTLEISTUNGEN    1komma6 - Multimediale Dienstleistungen GmbH
1X2 TECHNOLOGIES                                                  1x2 Technologies GmbH
4 SCREEN                                                                        4screen
                                                               ...                     
ZENJOB                                                                           ZENJOB
ZERO TO ONE SEARCH   RECRUITMENT AGENCY         Zero to One Search | Recruitment Agency
ZERTIFICON                                                    Zertificon Solutions GmbH
ZURICH INSURANCE                                                       Zurich Insurance
ZVOOVE                                                                           zvoove
Name: company,

In [21]:
# Map original company name to all listings from the same company.
df["company_clean"] = df["company_clean_ish"].map(company_cleaning_series)
df.filter(like="company")

,company,company_clean_ish,company_clean
0,Visa,VISA,Visa
1,Xsolla,XSOLLA,Xsolla
2,Global Market Solutions - Germany,GLOBAL MARKET SOLUTIONS GERMANY,Global Market Solutions - Germany
3,MeloTech,MELOTECH,MeloTech
4,vent.io,VENT IO,vent.io
...,...,...,...
1620,VidLab7 GmbH,VIDLAB7,VidLab7 GmbH
1621,Planet A Foods GmbH,PLANET A FOODS,Planet A Foods GmbH
1622,Lipotype GmbH,LIPOTYPE,Lipotype GmbH
1623,Amfileon AG,AMFILEON,Amfileon AG


In [22]:
# Check result.
df.loc[df["company_clean_ish"].str.contains("HUK"), ["company", "company_clean_ish", "company_clean"]]

,company,company_clean_ish,company_clean
7,HUK-COBURG,HUK COBURG,HUK-COBURG
188,HUK-COBURG,HUK COBURG,HUK-COBURG
251,HUK-COBURG,HUK COBURG,HUK-COBURG
421,HUK-COBURG,HUK COBURG,HUK-COBURG
481,HUK-COBURG,HUK COBURG,HUK-COBURG
571,HUK-COBURG,HUK COBURG,HUK-COBURG
933,HUK-COBURG Versicherungsgruppe,HUK COBURG,HUK-COBURG
987,HUK-COBURG Versicherungsgruppe,HUK COBURG,HUK-COBURG
1014,HUK-COBURG Versicherungsgruppe,HUK COBURG,HUK-COBURG
1050,HUK-COBURG Versicherungsgruppe,HUK COBURG,HUK-COBURG


## Job Titles

### Clean titles first

In [23]:
df["title"].value_counts()

title
Data Scientist (m/w/d)               48
Senior Data Scientist (m/w/d)        20
Senior Data Scientist                18
Data Scientist (w/m/d)               18
Data Scientist (all genders)         14
                                     ..
Data Engineer / Analyst               1
Head of / VP - Data Experience        1
Internship Program Autumn 2025        1
Trainee Analytics Engineer            1
Senior Regional Marketing Manager     1
Name: count, Length: 1163, dtype: int64

Remove gender (non-)specification, e.g. (m/w/d), (m/f/d), (m/ f/ d), (gn),...

In [24]:
# Define regular expression.
my_regex = r"\(?\s?\w\s?[/|,]\s?\w\s?[/|,]\s?\w+\s?\)?"
# .         - include last character before match (likely a blank)
# \(?       - optional opening paranthesis (some note "(m/w/d)", some e.g. "f/m/d")
# \s?\w\s?  - word characters surrounded by optional spaces (e.g. "(m / w / d)")
# [/|,]     - matches all present separators, e.g.: "m|w|d", "f/m/d", "m,w,div"
# \s?\w+\s? - additional '+' to match "m/w/div" at third position

# Test regex:
df.loc[df["title"].str.find("/") >= 0, "title"].str.replace(my_regex, "", regex=True).to_list()[:10]

['Data Scientist ',
 'Junior Data Scientist / AI Developer ',
 'Junior Engineer Machine Learning / Deep Learning ',
 'Data Scientist / ML Engineer ',
 'Junior Data Scientist  - Tableau',
 'Data Scientist ',
 'Data Scientist in Energy Tech Start Up - hybrid Berlin based ',
 'Battery Scientist ',
 'AI&Business Analyst / Consultant (Remote Internship)',
 'Data Scientist ']

In [25]:
# Apply string replace
df["title_clean"] = df["title"].str.replace(my_regex, "", regex=True)

In [26]:
# Check for additional spellings/specifications
df.loc[df["title"].str.contains("\(|\*"), ["title", "title_clean"]].sample(10)
# -> " (all genders)"; "(aGw*)"; "(gn)"

,title,title_clean
1615,Machine Learning Engineer (LLM/RAG),Machine Learning Engineer (LLM/RAG)
801,Genomics Data Scientist (f/m/x),Genomics Data Scientist
146,"Product Manager, DashOps (f/m/x)","Product Manager, DashOps"
210,Working Student (part-time) or Internship (ful...,Working Student (part-time) or Internship (ful...
1552,SAP Inhouse (Senior) Consultant in Business An...,SAP Inhouse (Senior) Consultant in Business An...
346,Manufacturing Data Analyst @ Deltia (Remote wi...,Manufacturing Data Analyst @ Deltia (Remote wi...
1544,Big Data Architect (m/w/d),Big Data Architect
1410,AI (Agent) Engineer (f/m/d) | Jobs at VisionAI...,AI (Agent) Engineer | Jobs at VisionAI - Jobs
679,Product Analyst – Business Data & Insights (m/...,Product Analyst – Business Data & Insights
120,Bachelor of Science (m/w/d) Data Science und K...,Bachelor of Science Data Science und Künstlic...


In [27]:
df.loc[df["title_clean"].str.contains("all gender|gn|aGw", regex=True), "title_clean"].to_list()

['Data Scientist (all gender)',
 'Working Student AI/ML (all genders) - (Any location in Germany)',
 'Working Student Data Science and Data Engineering (all genders!)',
 'Data Scientist - Customer Data Platform (all genders)',
 'Head of AI (all genders)',
 "Aignostics' Talent Community",
 'GOglobal Trainee Data & AI (all genders)',
 'Working Student PV Performance Engineering (all genders)',
 'Strategy Manager Life Science (all genders)',
 'Werkstudierende - Tutoring im Data Science (all genders)',
 'Junior AI Platform Engineer or Working Student (all genders)',
 'Data Scientist - Time Estimations, Logistics (all genders)',
 'Working Student Aerospace Data Science (all genders)',
 '(Senior) BI Analyst (all genders)',
 'Data Analyst (all gender) – Process Mining und Datenanalyse',
 'Data Scientist  - Sektion für Hämatologische Spezialdiagnostik',
 'DWH Analyst (all genders)',
 'Data Analyst (all genders)',
 'Trainee Data Engineering (all genders)',
 'Head of Data (all genders)',
 'Data 

In [28]:
# Define and test regular expression
my_regex2 = r"\((\baGw.?|\bgn.?|\ball genders?.?)\)" # "(gn)", (gn*)", "(all gender)", "(all genders)", "(all genders!)"
df.loc[df["title_clean"].str.contains("all gender|gn|aGw", regex=True), "title_clean"].str.replace(my_regex2, "", regex=True).to_list()

['Data Scientist ',
 'Working Student AI/ML  - (Any location in Germany)',
 'Working Student Data Science and Data Engineering ',
 'Data Scientist - Customer Data Platform ',
 'Head of AI ',
 "Aignostics' Talent Community",
 'GOglobal Trainee Data & AI ',
 'Working Student PV Performance Engineering ',
 'Strategy Manager Life Science ',
 'Werkstudierende - Tutoring im Data Science ',
 'Junior AI Platform Engineer or Working Student ',
 'Data Scientist - Time Estimations, Logistics ',
 'Working Student Aerospace Data Science ',
 '(Senior) BI Analyst ',
 'Data Analyst  – Process Mining und Datenanalyse',
 'Data Scientist  - Sektion für Hämatologische Spezialdiagnostik',
 'DWH Analyst ',
 'Data Analyst ',
 'Trainee Data Engineering ',
 'Head of Data ',
 'Data Scientist ',
 'Data Scientist ',
 'Research Assistant, Molecular Diagnostics ',
 'Staff Data Scientist - Ad Tech ',
 'Data Analyst Battery Cell Testing ',
 'Senior Data Scientist - Time Estimations, Logistics ',
 'BI Analyst ',
 'Int

In [29]:
# Apply regex2
df["title_clean"] = df["title_clean"].str.replace(my_regex2, "", regex=True)

In [30]:
# Clear white space
df["title_clean"] = df["title_clean"].str.strip()

In [31]:
df["title_clean"].value_counts()

title_clean
Data Scientist                                                        115
Senior Data Scientist                                                  59
Data Analyst                                                           25
Data Engineer                                                          21
Machine Learning Engineer                                              17
                                                                     ... 
Business Intelligence Engineer*                                         1
Head of IoT                                                             1
Working student software development                                    1
Wissenschaftliche/r Mitarbeiter/in  | Thermomechanische Simulation      1
Senior Regional Marketing Manager                                       1
Name: count, Length: 1081, dtype: int64

In [32]:
np.sort(df["title_clean"].unique()).tolist()

['(Advanced) Data Developer',
 '(Associate) Consultant (Gen) AI & Data Scientist | AI Garage',
 '(Cloud) Data Engineer / BI Developer',
 '(Global) Senior Applied Research Scientist',
 '(Junior) AI Engineer',
 '(Junior) Berater Business Intelligence',
 '(Junior) Cloud Data Engineer',
 '(Junior) Consultant Data Analytics',
 '(Junior) Consultant Data Science & Artificial Intelligence',
 '(Junior) Consultant | Gen AI Finance',
 '(Junior) Data Analyst/ Business Analyst Procurement',
 '(Junior) Data Engineer',
 '(Junior) Data Product Associate',
 '(Junior) Data Scientist',
 '(Junior) Digital Produktmanager  AurumCars',
 '(Junior) Digital Produktmanager  Mietwagen',
 '(Junior) Digital Produktmanager  Payment Mietwagen',
 '(Junior) Online-Marketing Produktmanager  Mietwagen',
 '(Junior) Product Manager Financial Services',
 '(Junior) Produktmanager  Chatbot Mietwagen',
 '(Junior) Produktmanager  Operations Analytics Mietwagen',
 '(Junior) Produktmanager  SEO-Automatisierung Mietwagen',
 '(Juni

## Seniority Level

In [33]:
# The level of seniority will be extracted from information provided in the raw job titles.
#  E.g.: "(Junior) AI Engineer", "Senior Data Scientist"
np.sort(df["title"].unique()).tolist()

['(Advanced) Data Developer (m/w/d)',
 '(Associate) Consultant (Gen) AI & Data Scientist | AI Garage (m/w/d)',
 '(Cloud) Data Engineer / BI Developer (m/w/d) (m/w/d)',
 '(Global) Senior Applied Research Scientist',
 '(Junior) AI Engineer (m/w/d)',
 '(Junior) Berater Business Intelligence (m/w/d)',
 '(Junior) Cloud Data Engineer (all genders)',
 '(Junior) Consultant Data Analytics (m/w/d)',
 '(Junior) Consultant Data Science & Artificial Intelligence (m/w/d)',
 '(Junior) Consultant | Gen AI Finance (m/w/d)',
 '(Junior) Data Analyst/ Business Analyst Procurement',
 '(Junior) Data Engineer (m/w/d)',
 '(Junior) Data Engineer (m/w/x)',
 '(Junior) Data Product Associate (f/m/x)',
 '(Junior) Data Scientist',
 '(Junior) Digital Produktmanager (m/w/d) AurumCars',
 '(Junior) Digital Produktmanager (m/w/d) Mietwagen',
 '(Junior) Digital Produktmanager (m/w/d) Payment Mietwagen',
 '(Junior) Online-Marketing Produktmanager (m/w/d) Mietwagen',
 '(Junior) Product Manager Financial Services (d/w/m)',


In [34]:
# Some job platforms already provided the information of seniority level
df["seniority_level"].value_counts(dropna=False)

seniority_level
NaN                  829
Mid-Senior level     269
Entry level          250
Associate            100
Internship            96
Not Applicable        58
Director              16
Executive              7
Name: count, dtype: int64

In [35]:
df["seniority_level"] = df["seniority_level"].str.strip()

In [36]:
# Define search words and associated seniority
seniority_dict = {
    "Senior": "Senior",
    "Sen.": "Senior",
    "Sr.": "Senior",
    "(Advanced)": "Senior",
    "Teamlead": "Senior",

    "Junior": "Junior",
    "Jun.": "Junior",
    "Jr.": "Junior",
    "Trainee": "Trainee",
    "Internship": "Intern",
    "Intern": "Intern",
    "Praktikum": "Intern",
    "PRAKTIKUM": "Intern",
    "Praktikant": "Intern",

    "Head": "Director",
    "Lead": "Director",
    "Principal": "Director",
    "Director": "Director",

    "Abschlussarbeit": "Student", 
    "Working student": "Student", 
    "Working Student": "Student", 
    "Werkstudent": "Student", 
    "WERKSTUDENT": "Student", 
    "Doktorand": "Student", 
    "Doctoral": "Student", 
    "Bachelor": "Student",
    "Studium": "Student",

    "Post Doc": "Postdoc",
    "Postdoc": "Postdoc", 
    "PostDoc": "Postdoc",
    "Wissenschaftliche/r Mitarbeiter/in": "Postdoc",
    "Wissenschaftlicher Mitarbeiter": "Postdoc",
}

df["seniority_level_new"] = pd.NA

for search_term, seniority in seniority_dict.items():
    df.loc[df["title"].str.contains(search_term, regex=False), "seniority_level_new"] = seniority


In [37]:
df.loc[df["seniority_level_new"].isna() == False, ["title", "seniority_level_new", "seniority_level"]]

,title,seniority_level_new,seniority_level
1,Junior Data Scientist,Junior,Entry level
3,Data Scientist Intern,Intern,Entry level
6,Junior Data Scientist / AI Developer (m/w/d),Junior,Entry level
7,Junior Engineer Machine Learning / Deep Learni...,Junior,Associate
10,Junior Data Scientist (w/m/d) - Tableau,Junior,Entry level
...,...,...,...
1618,Senior Database Engineer (m/w/d) (Data Engineer),Senior,NaN
1619,(Senior) Data Engineer (m/w/d) (Informatiker/in),Senior,NaN
1620,Senior Machine Learning Engineer,Senior,NaN
1623,Senior Quant Developer (f/m/d),Senior,NaN


In [38]:
# Check listings which did not get categorized by code above.
np.sort(df.loc[df["seniority_level_new"].isna()]["title"].unique())
# -> Looks ok; no specification regarding seniority provided in title.

array(['(Associate) Consultant (Gen) AI & Data Scientist | AI Garage (m/w/d)',
       '(Cloud) Data Engineer / BI Developer (m/w/d) (m/w/d)',
       '(Wirtschafts-)Mathematiker, Statistiker, Data Scientist für die Tarifentwicklung Komposit',
       '1D/2D MATERIAL SCIENTIST (M/F/D)',
       '2025 Software Development Engineer - Machine Learning (m/w/d)',
       'ACTUARIAL DATA SCIENTIST (M/W/D)', 'AI & Data - Analyst',
       'AI (Agent) Engineer (f/m/d) | Jobs at VisionAI - Jobs',
       'AI Architect & Optimization Specialist (m/w/d)',
       'AI Architect / Machine Learning Engineer (m,f,d)',
       'AI Cloud Engineer', 'AI Consultant', 'AI Developer',
       'AI Developer (m/w/d)', 'AI Engineer', 'AI Engineer (m/w/d)',
       'AI Engineer - Germany (Remote)',
       'AI Engineer / AI Developer (m/w/d) bei einem SaaS-Unternehmen aus Würzburg (Full-Remote oder vor Ort)',
       'AI Engineer / Data Scientist m|w|d',
       'AI Engineer // Data Scientist (all genders)',
       'AI Engi

In [39]:
# Set remaining unclassified listings to "Assiciate/Mid-Level"
df.loc[df["seniority_level_new"].isna(), "seniority_level_new"] = "Associate/Mid-Level"

In [40]:
df["seniority_level_new"].value_counts()

seniority_level_new
Associate/Mid-Level    926
Senior                 299
Student                109
Director                84
Junior                  80
Intern                  80
Postdoc                 25
Trainee                 22
Name: count, dtype: int64

#### Compare seniority levels with those provided on the job platforms

In [41]:
df["seniority_level"].unique()

array(['Associate', 'Entry level', 'Mid-Senior level', 'Not Applicable',
       'Internship', 'Director', 'Executive', nan], dtype=object)

In [42]:
np.sort(df.loc[df["seniority_level"] == "Mid-Senior level", "title"]).tolist()
# some junior positions

['(Global) Senior Applied Research Scientist',
 '(Junior) AI Engineer (m/w/d)',
 '(Junior) Data Engineer (m/w/d)',
 '(Junior) Data Engineer (m/w/x)',
 '(Junior) Data Product Associate (f/m/x)',
 '(Senior) Data Analyst / Data Scientist (f/m/d) in Data Division',
 '(Senior) Data Scientist (Analytics), Merchant',
 '(Senior) Data Scientist / ML Engineer (m/w/d)',
 '(Senior) Machine Learning Data Scientist (d/f/m)',
 '(Senior) Machine Learning Engineer (m/f/d)',
 '2025 Software Development Engineer - Machine Learning (m/w/d)',
 'AI Consultant',
 'AI Engineer / AI Developer (m/w/d) bei einem SaaS-Unternehmen aus Würzburg (Full-Remote oder vor Ort)',
 'AI Engineer // Data Scientist (all genders)',
 'AI Lead',
 'AI Platform Engineer (all genders)',
 'AI Researcher',
 'AI consultant  - Energy',
 'AI\xa0computational\xa0\xa0material\xa0\xa0researcher/expert',
 'Analyst - Fokus Data Science / AI (m/w/d)',
 'Analytics Engineer (m/f/d)',
 'Analytics Engineering Lead',
 'Applied Scientist for Probab

In [43]:
np.sort(df.loc[df["seniority_level"] == "Associate", "title"]).tolist()
# Some senior, some working student

['(Senior) BI Analyst (all genders)',
 '(Senior) Consultant Data Science & MLOps, d/f/m',
 'AI Consultant',
 'AI Developer (m/w/d)',
 'AI Engineer (Senior)',
 'AI Engineer (m/w/d)',
 'AI Software Developer in Test (f/m/d)',
 'Analyst - Fokus Data Science / AI (m/w/d)',
 'Analytics Engineer (m/w/d)',
 'Artificial Intelligence Engineer',
 'Artificial Intelligence Engineer',
 'Artificial Intelligence Specialist',
 'Artificial Intelligence Specialist (w/m/d)',
 'Associate - Index Operations (m/f/d)',
 'Associate / (Senior) AI Technology Consultant (m/w/d)',
 'BI & Data Modeling Expert (m/f/d)',
 'Bench Scientist (all genders)',
 'Breeding Informatics Analyst (d/f/m) Full Time - in Tübingen',
 'Business Intelligence Developer (m/w/d) Tech / E-Commerce',
 'CRA/SCRA',
 'DWH Analyst (all genders)',
 'Data & AI Business Consultant (m/w/d)',
 'Data & AI Strategist (m/f/d)',
 'Data Analyst',
 'Data Analyst (m/f/d) in IT security company',
 'Data Analyst (m/w/d)',
 'Data Analyst (m/w/d)',
 'Data A

In [44]:
filter_condition = (df["seniority_level"] != df["seniority_level_new"]) \
    & (df["seniority_level"].isna() == False)

df.loc[filter_condition, ["seniority_level", "seniority_level_new"]]

,seniority_level,seniority_level_new
0,Associate,Associate/Mid-Level
1,Entry level,Junior
2,Mid-Senior level,Associate/Mid-Level
3,Entry level,Intern
4,Associate,Associate/Mid-Level
...,...,...
791,Associate,Senior
792,Mid-Senior level,Associate/Mid-Level
793,Mid-Senior level,Senior
794,Mid-Senior level,Director


#### Conclusion

Rather inconclusive... However, seniority levels will likely match closer if entry level seniority levels are further grouped

#### Create column with 5 groups

In [45]:
# Group entry level positions together
df["seniority_level_5"] = df["seniority_level_new"]

entry_positions = ["Junior", "Intern", "Student", "Trainee"]

df.loc[df["seniority_level_new"].isin(entry_positions), "seniority_level_5"] = "Entry level"

In [46]:
df["seniority_level_5"].value_counts()

seniority_level_5
Associate/Mid-Level    926
Senior                 299
Entry level            291
Director                84
Postdoc                 25
Name: count, dtype: int64

In [47]:
filter_condition = (df["seniority_level"] != df["seniority_level_5"]) \
    & (df["seniority_level"].isna() == False)

df.loc[filter_condition, ["seniority_level", "seniority_level_new", "seniority_level_5"]]

,seniority_level,seniority_level_new,seniority_level_5
0,Associate,Associate/Mid-Level,Associate/Mid-Level
2,Mid-Senior level,Associate/Mid-Level,Associate/Mid-Level
4,Associate,Associate/Mid-Level,Associate/Mid-Level
5,Mid-Senior level,Associate/Mid-Level,Associate/Mid-Level
7,Associate,Junior,Entry level
...,...,...,...
790,Internship,Student,Entry level
791,Associate,Senior,Senior
792,Mid-Senior level,Associate/Mid-Level,Associate/Mid-Level
793,Mid-Senior level,Senior,Senior


## Detect and Drop Duplicates

First, I'll check whether duplicates can be detected by using the entirety or the most relevant columns.

These include job title, company name, seniority and location.

Why? The same job could be advertised for different branches/locations.
A similar job posting might have been posted targeting different levels of seniority

I'll also use the job description, however, this will likely not proof to be efficient as simple matching fails 

even if one character differs. This could be a "\n" or an additional heading, e.g. "Benefits:

Hence, string similarity meausures will be applied in the upcoming steps as well.

In [48]:
# See whether there are duplicates when taking into account all columns
df[df.duplicated(keep=False)].sort_values(by="company")

,title,company,location,salary,seniority_level,date_posted,date_accessed,days_online,number_applicants,job_type_Vollzeit,...,job_type_Praktikum,job_type_Werkstudent,job_description_anonymized,source_id,location_clean,company_clean_ish,company_clean,title_clean,seniority_level_new,seniority_level_5
955,"Applied Scientist, Artificial General Intellig...",Amazon Development Center Germany GmbH,Berlin,NaN,NaN,NaN,2024-12-12,NaN,NaN,1,...,0,0,"PhD, or a Master's degree and experience in CS...",2,Berlin,AMAZON DEVELOPMENT CENTER GERMANY,Amazon Development Center Germany GmbH,"Applied Scientist, Artificial General Intellig...",Associate/Mid-Level,Associate/Mid-Level
1032,"Applied Scientist, Artificial General Intellig...",Amazon Development Center Germany GmbH,Berlin,NaN,NaN,NaN,2024-12-12,NaN,NaN,1,...,0,0,"PhD, or a Master's degree and experience in CS...",2,Berlin,AMAZON DEVELOPMENT CENTER GERMANY,Amazon Development Center Germany GmbH,"Applied Scientist, Artificial General Intellig...",Associate/Mid-Level,Associate/Mid-Level
1076,Senior Consultant IT Regulatory Risk / Risikom...,Deloitte GmbH,Köln,NaN,NaN,NaN,2024-12-12,NaN,NaN,1,...,0,0,Du machst den Unterschied. Ob im Business Audi...,2,Köln,DELOITTE,Deloitte,Senior Consultant IT Regulatory Risk / Risikom...,Senior,Senior
1077,Senior Consultant IT Regulatory Risk / Risikom...,Deloitte GmbH,Köln,NaN,NaN,NaN,2024-12-12,NaN,NaN,1,...,0,0,Du machst den Unterschied. Ob im Business Audi...,2,Köln,DELOITTE,Deloitte,Senior Consultant IT Regulatory Risk / Risikom...,Senior,Senior
1178,Data Architekt / Data Engineer / Data Scientis...,Wego Systembaustoffe GmbH,Hanau,NaN,NaN,NaN,2024-12-12,NaN,NaN,0,...,0,0,für unsere Zentrale in Hanau-Steinheim\nDeine ...,2,Hanau,WEGO SYSTEMBAUSTOFFE,Wego Systembaustoffe GmbH,Data Architekt / Data Engineer / Data Scientist,Associate/Mid-Level,Associate/Mid-Level
1237,Data Architekt / Data Engineer / Data Scientis...,Wego Systembaustoffe GmbH,Hanau,NaN,NaN,NaN,2024-12-12,NaN,NaN,0,...,0,0,für unsere Zentrale in Hanau-Steinheim\nDeine ...,2,Hanau,WEGO SYSTEMBAUSTOFFE,Wego Systembaustoffe GmbH,Data Architekt / Data Engineer / Data Scientist,Associate/Mid-Level,Associate/Mid-Level


In [49]:
# Filter to only a few selected columns
columns_selected = ["title_clean", 
                    "company_clean", 
                    "location_clean", 
                    "seniority_level_5", 
                    "job_description_anonymized"]
sum(df.filter(columns_selected).duplicated())

11

In [50]:
# Show those duplicate listings.
df[df.filter(columns_selected).duplicated(keep=False)] \
    .filter(columns_selected) \
    .sort_values(by=["title_clean", "location_clean"])

,title_clean,company_clean,location_clean,seniority_level_5,job_description_anonymized
955,"Applied Scientist, Artificial General Intellig...",Amazon Development Center Germany GmbH,Berlin,Associate/Mid-Level,"PhD, or a Master's degree and experience in CS..."
1032,"Applied Scientist, Artificial General Intellig...",Amazon Development Center Germany GmbH,Berlin,Associate/Mid-Level,"PhD, or a Master's degree and experience in CS..."
710,DATA ANALYST,Instaffo,München,Associate/Mid-Level,Du bist interessiert an der Stelle als DATA AN...
716,DATA ANALYST,Instaffo,München,Associate/Mid-Level,Du bist interessiert an der Stelle als DATA AN...
536,Data Analyst,CBTW,Frankfurt am Main,Associate/Mid-Level,Willkommen bei CBTW! Wir sind ein innovatives ...
599,Data Analyst,CBTW,Frankfurt am Main,Associate/Mid-Level,Willkommen bei CBTW! Wir sind ein innovatives ...
682,Data Analyst,CBTW,Frankfurt am Main,Associate/Mid-Level,Willkommen bei CBTW! Wir sind ein innovatives ...
609,Data Analyst,CBTW,München,Associate/Mid-Level,Willkommen bei CBTW! Wir sind ein innovatives ...
666,Data Analyst,CBTW,München,Associate/Mid-Level,Willkommen bei CBTW! Wir sind ein innovatives ...
1178,Data Architekt / Data Engineer / Data Scientist,Wego Systembaustoffe GmbH,Hanau,Associate/Mid-Level,für unsere Zentrale in Hanau-Steinheim\nDeine ...


In [51]:
# These are clearly duplicates. Drop the duplicate row; i.e. Keep the first occurance of these listings
columns_selected = ["title_clean", 
                    "company_clean", 
                    "location_clean", 
                    "seniority_level_5", 
                    "job_description_anonymized"]

print("Columns before removing duplicates:", len(df))
df = df.drop_duplicates(subset=columns_selected)

print("Columns after removing duplicates:", len(df))

Columns before removing duplicates: 1625
Columns after removing duplicates: 1614


### Refine columns to detect duplicates

In [52]:
columns_selected = ["title_clean", 
                    "company_clean", 
                    "location_clean", 
                    "seniority_level_5"]

sum(df.filter(columns_selected).duplicated())

146

In [53]:
df.columns

Index(['title', 'company', 'location', 'salary', 'seniority_level',
       'date_posted', 'date_accessed', 'days_online', 'number_applicants',
       'job_type_Vollzeit', 'job_type_Teilzeit', 'job_type_Festanstellung',
       'job_type_BefristeterVertrag', 'job_type_Homeoffice',
       'job_type_FreieMitarbeit', 'job_type_Traineeprogramm',
       'job_type_Berufsausbildung', 'job_type_Praktikum',
       'job_type_Werkstudent', 'job_description_anonymized', 'source_id',
       'location_clean', 'company_clean_ish', 'company_clean', 'title_clean',
       'seniority_level_new', 'seniority_level_5'],
      dtype='object')

In [54]:
# Perform some manual checks in excel file
columns_selected = ["title_clean", 
                    "company_clean", 
                    "location_clean", 
                    "seniority_level_5"]

display_columns = columns_selected + ["source_id", "job_description_anonymized"]

row_selection = df.filter(columns_selected).duplicated(keep=False)

df_out = df.loc[row_selection, display_columns] \
    .sort_values(by=["company_clean", "title_clean", "location_clean"])
df_out

,title_clean,company_clean,location_clean,seniority_level_5,source_id,job_description_anonymized
154,Data Scientist,ACONEXT,Fellbach,Associate/Mid-Level,1,Willst du die InnovatiWEITERE STELLENANGEBOTES...
823,Data Scientist,ACONEXT,Fellbach,Associate/Mid-Level,2,Willst du die Innovationen voe\nBewerbung WEIT...
1288,Data Scientist,ACONEXT,Fellbach,Associate/Mid-Level,3,MITTELSTÄNDISCH • PRAGMATISCH • LEIDENSCHAFTLI...
163,Data Scientist Automotive,ACONEXT,Fellbach,Associate/Mid-Level,1,Willst du die Innovationen vog WEITERE STELLEN...
809,Data Scientist Automotive,ACONEXT,Fellbach,Associate/Mid-Level,2,Willst du die Innovationen von morgen mitewerb...
...,...,...,...,...,...,...
1239,Trainee Data Science - 12 Monate,taod Consulting GmbH,Köln,Entry level,2,Wir suchen dataful minds. Mit unserer Leidensc...
301,Data Scientist - Ranking,trivago,Düsseldorf,Associate/Mid-Level,1,"When travelers are searching for a hotel, we w..."
839,Data Scientist - Ranking,trivago,Düsseldorf,Associate/Mid-Level,2,"When travelers are searching for a hotel, we w..."
1225,Senior Data Scientist * Generative AI,valantic Software & Technology Innovations GmbH,Eschborn,Senior,2,Beschreibung\nWir suchen einen Senior Data Sci...


In [55]:
# Store original index for upcoming aggregation
df["index_pre_agg"] = df.index
df["index_agg_list"] = df.index # to store all ids that were aggregated into 1 row

In [56]:
group_by_columns = ["title_clean", "company_clean", "location_clean"]

# set aggregation functions to: 
# - max for numeric values (except index), 
# - first for object types (except source, which will be stored in a set)
aggregation_functions = {col: "first" if df[col].dtype == "O" else "max" for col in df.columns}
aggregation_functions.update(index_pre_agg = min)
aggregation_functions.update(index_agg_list = list)
aggregation_functions.update(source_id = list)

df_len_pre = len(df)
print("number of rows before aggregation:", df_len_pre)

# Aggregate; Restore original index and order
df = df.groupby(group_by_columns, as_index=False).agg(aggregation_functions).reset_index(drop=True)
df.index = df["index_pre_agg"].to_list()
df = df.sort_index()

print(f"number of rows after aggregation: {len(df)} (dif: {df_len_pre - len(df)})")

number of rows before aggregation: 1614
number of rows after aggregation: 1468 (dif: 146)


In [57]:
df.head()

,title,company,location,salary,seniority_level,date_posted,date_accessed,days_online,number_applicants,job_type_Vollzeit,...,job_description_anonymized,source_id,location_clean,company_clean_ish,company_clean,title_clean,seniority_level_new,seniority_level_5,index_pre_agg,index_agg_list
0,Data Science Graduate,Visa,Berlin,None,Associate,2024-11-22,2024-12-09,17.0,200.0,1,...,Company DescriptionFascinated by the future an...,[1],Berlin,VISA,Visa,Data Science Graduate,Associate/Mid-Level,Associate/Mid-Level,0,[0]
1,Junior Data Scientist,Xsolla,Berlin,None,Entry level,2024-11-30,2024-12-09,9.0,200.0,1,...,Xsolla is looking for a Junior Data Scientist ...,"[1, 2]",Berlin,XSOLLA,Xsolla,Junior Data Scientist,Junior,Entry level,1,"[1, 1183]"
2,Data Scientist,Global Market Solutions - Germany,Frankfurt am Main,None,Mid-Senior level,2024-11-13,2024-12-09,26.0,200.0,1,...,GMS is an operational consultancy with experti...,"[1, 1]",Frankfurt am Main,GLOBAL MARKET SOLUTIONS GERMANY,Global Market Solutions - Germany,Data Scientist,Associate/Mid-Level,Associate/Mid-Level,2,"[2, 19]"
3,Data Scientist Intern,MeloTech,Berlin,None,Entry level,2024-01-31,2024-12-09,313.0,200.0,1,...,Your missionPropose innovative solutions by ex...,"[1, 2]",Berlin,MELOTECH,MeloTech,Data Scientist Intern,Intern,Entry level,3,"[3, 1105]"
4,Data Scientist (f/m/d),vent.io,Frankfurt am Main,None,Associate,2024-11-20,2024-12-09,19.0,200.0,1,...,The target of vent.io is to accelerate the gro...,[1],Frankfurt am Main,VENT IO,vent.io,Data Scientist,Associate/Mid-Level,Associate/Mid-Level,4,[4]


In [58]:
# Transform source into string again.
df["source_id"].value_counts()

source_id
[1]          683
[2]          340
[3]          306
[1, 2]        70
[2, 3]        29
[2, 2]        13
[1, 3]        11
[1, 1]         9
[1, 2, 3]      4
[1, 2, 2]      2
[2, 2, 3]      1
Name: count, dtype: int64

### Flag duplicates using job descriptions

The Levenshtein algorithm is used as it is recommended for comparing strings of different lenghts.
 Also, in a preceeding test run, the similarity ratios were calculated around 20 times faster
 compared to the SequenceMatcher ratio applied by the difflib package (Lev: 2.4 sec, difflib: 55.3 sec)

Rows will be flagged if similarity score is larger than pre-defined thresholds, and location as well as seniority are not identical.
(As mentioned before, one job could be listed targeting different seniority levels or locations)
Store/concatenate indices in both listings to compare in later stage.

In [59]:
df_indices = df.index.to_list()

# Define thresholds to be flagged
similarity_thresholds = [0.90, 0.93, 0.95, 0.97]

# Initialize dictionary to store results in
col_names_suffix = [str(f"{s:.2f}").replace(".", "_") for s in similarity_thresholds]
col_names = [f"similarity_results_{t}" for t in col_names_suffix]

similarity_results = pd.DataFrame(
        {col: [[] for _ in range(len(df))] for col in col_names},
        index=df_indices
    )

# Initialize iterator to be used to flag multiple listings 
merge_group_iterator = 0

for ind_nr, index1 in enumerate(df_indices):
    if ind_nr > 0 and ind_nr % 100 == 0:
        print(f"Now comparing at index1: {ind_nr}/{len(df)}")
    loc1 = df["location_clean"][index1]
    sen1 = df["seniority_level_new"][index1]
    str1 = df["job_description_anonymized"][index1]

    for index2 in df_indices:
        # Skip comparison if rows have already been compared
        if index1 >= index2:
            continue
        # Skip comparison if location is different.
        loc2 = df["location_clean"][index2]
        if loc1 != loc2:
            continue
        # Skip comparison if seniority level is different.
        sen2 = df["seniority_level_new"][index2]
        if sen1 != sen2:
            continue
        
        # Compute similarity
        str2 = df["job_description_anonymized"][index2]
        similarity = Levenshtein.ratio(str1, str2)
        
        # Check value against each threshold and store results in respective cells
        for threshold_id, threshold in enumerate(similarity_thresholds):
            if similarity >= threshold:
                # print(f"Similar results found at index1: {index1}, index2: {index2} (threshold: {threshold}).")
                col = col_names[threshold_id]
                # Use merge_group if existing, otherwise grab from iterator
                if len(similarity_results.loc[index1, col]) > 0:
                    merge_group = similarity_results.loc[index1, col][-1][-1]
                elif len(similarity_results.loc[index2, col]) > 0:
                    merge_group = similarity_results.loc[index2, col][-1][-1]
                else:
                    merge_group = merge_group_iterator
                    merge_group_iterator += 1
                    
                sim1 = tuple([index2, round(similarity, 3), merge_group])
                sim2 = tuple([index1, round(similarity, 3), merge_group])
                
                similarity_results[col][index1].append(sim1)
                similarity_results[col][index2].append(sim2)

# this took 8:22 minutes
# this took 0:38 minutes (with location check)
# this took 0:18 minutes (with location and seniority check)

Now comparing at index1: 100/1468
Now comparing at index1: 200/1468
Now comparing at index1: 300/1468
Now comparing at index1: 400/1468
Now comparing at index1: 500/1468
Now comparing at index1: 600/1468
Now comparing at index1: 700/1468
Now comparing at index1: 800/1468
Now comparing at index1: 900/1468
Now comparing at index1: 1000/1468
Now comparing at index1: 1100/1468
Now comparing at index1: 1200/1468
Now comparing at index1: 1300/1468
Now comparing at index1: 1400/1468


In [60]:
# similarity_results["similarity_results_0_90"].map(lambda x: len(x)==0, pd.NA).isna().sum()
print(similarity_results["similarity_results_0_90"].map(lambda x: len(x)>0).sum())
print(similarity_results["similarity_results_0_93"].map(lambda x: len(x)>0).sum())
print(similarity_results["similarity_results_0_95"].map(lambda x: len(x)>0).sum())
print(similarity_results["similarity_results_0_97"].map(lambda x: len(x)>0).sum())

# CHECKS                             0_90   0_93    0_95    0_97
# Without location check:            315    -       265     229
# With location check:               153    -       97      75
# With location and seniority check: 125    94      86      69

125
94
86
69


In [61]:
# merge results to dataframe
df = df.merge(similarity_results, left_index=True, right_index=True)
df.head()

,title,company,location,salary,seniority_level,date_posted,date_accessed,days_online,number_applicants,job_type_Vollzeit,...,company_clean,title_clean,seniority_level_new,seniority_level_5,index_pre_agg,index_agg_list,similarity_results_0_90,similarity_results_0_93,similarity_results_0_95,similarity_results_0_97
0,Data Science Graduate,Visa,Berlin,None,Associate,2024-11-22,2024-12-09,17.0,200.0,1,...,Visa,Data Science Graduate,Associate/Mid-Level,Associate/Mid-Level,0,[0],[],[],[],[]
1,Junior Data Scientist,Xsolla,Berlin,None,Entry level,2024-11-30,2024-12-09,9.0,200.0,1,...,Xsolla,Junior Data Scientist,Junior,Entry level,1,"[1, 1183]",[],[],[],[]
2,Data Scientist,Global Market Solutions - Germany,Frankfurt am Main,None,Mid-Senior level,2024-11-13,2024-12-09,26.0,200.0,1,...,Global Market Solutions - Germany,Data Scientist,Associate/Mid-Level,Associate/Mid-Level,2,"[2, 19]",[],[],[],[]
3,Data Scientist Intern,MeloTech,Berlin,None,Entry level,2024-01-31,2024-12-09,313.0,200.0,1,...,MeloTech,Data Scientist Intern,Intern,Entry level,3,"[3, 1105]",[],[],[],[]
4,Data Scientist (f/m/d),vent.io,Frankfurt am Main,None,Associate,2024-11-20,2024-12-09,19.0,200.0,1,...,vent.io,Data Scientist,Associate/Mid-Level,Associate/Mid-Level,4,[4],[],[],[],[]


In [62]:
print(len(df))
print(sum(df["similarity_results_0_90"].apply(lambda x: len(x)>0)))
print(sum(df["similarity_results_0_93"].apply(lambda x: len(x)>0)))
print(sum(df["similarity_results_0_95"].apply(lambda x: len(x)>0)))
print(sum(df["similarity_results_0_97"].apply(lambda x: len(x)>0)))

1468
125
94
86
69


In [63]:
# Extract merge group from tuple
def get_last_merge_group(x: tuple) -> int:
    if len(x) > 0: 
        return x[-1][-1]
    return pd.NA

df["similarity_results_0_90_mgroup"] = df["similarity_results_0_90"].apply(get_last_merge_group)
df["similarity_results_0_93_mgroup"] = df["similarity_results_0_93"].apply(get_last_merge_group)
df["similarity_results_0_95_mgroup"] = df["similarity_results_0_95"].apply(get_last_merge_group)
df["similarity_results_0_97_mgroup"] = df["similarity_results_0_97"].apply(get_last_merge_group)

In [64]:
df[df["similarity_results_0_97_mgroup"].isna() == False].filter(like="similarity_").head()

,similarity_results_0_90,similarity_results_0_93,similarity_results_0_95,similarity_results_0_97,similarity_results_0_90_mgroup,similarity_results_0_93_mgroup,similarity_results_0_95_mgroup,similarity_results_0_97_mgroup
57,"[(1228, 0.987, 0)]","[(1228, 0.987, 1)]","[(1228, 0.987, 2)]","[(1228, 0.987, 3)]",0,1,2,3
59,"[(256, 0.981, 4)]","[(256, 0.981, 5)]","[(256, 0.981, 6)]","[(256, 0.981, 7)]",4,5,6,7
78,"[(744, 0.919, 8), (1142, 0.979, 8)]","[(1142, 0.979, 9)]","[(1142, 0.979, 10)]","[(1142, 0.979, 11)]",8,9,10,11
82,"[(247, 1.0, 12), (424, 1.0, 12)]","[(247, 1.0, 13), (424, 1.0, 13)]","[(247, 1.0, 14), (424, 1.0, 14)]","[(247, 1.0, 15), (424, 1.0, 15)]",12,13,14,15
86,"[(1089, 0.976, 16)]","[(1089, 0.976, 17)]","[(1089, 0.976, 18)]","[(1089, 0.976, 19)]",16,17,18,19


#### Perform manual checks

In [65]:
#df.to_excel("TEMP_CHECK_CUPLIKATES_SAME_LOC_SAME_SEN_ALL4_MGROUPS2.xlsx", index=True, header=True)

In [66]:
# Having checked the data in excel, the performance of each threshold can be expressed in a confusion matrix.
#  (This is a very simplified approach, as listing were only swiftly checked.)
confusion_matrix = pd.DataFrame(
    {"090": {"FP": 26, "TN":  0, "FN":  0, "TP": 30+69},
     "093": {"FP":  9, "TN": 17, "FN": 14, "TP": 16+69},
     "095": {"FP":  7, "TN": 19, "FN": 18, "TP": 12+69},
     "097": {"FP":  0, "TN": 26, "FN": 30, "TP":  0+69}},
).T

confusion_matrix

,FP,TN,FN,TP
090,26,0,0,99
093,9,17,14,85
095,7,19,18,81
097,0,26,30,69


It is of no surprise that the more conservative threshold (0.97) does not falsely detect any duplicates, while the lowest threshold yields the most duplicates in absolute values, but also flags a lot of listings as duplicate which are in fact not duplicates.

In [67]:
# Copmute performance metrices based on data in confusion matrix
confusion_matrix["n"] = confusion_matrix.sum(axis=1)

confusion_matrix["precision"] = confusion_matrix["TP"] / (confusion_matrix["TP"] + confusion_matrix["FP"])
confusion_matrix["recall"] = confusion_matrix["TP"] / (confusion_matrix["TP"] + confusion_matrix["FN"])
confusion_matrix["f1"] = (2 * confusion_matrix["precision"] * confusion_matrix["recall"]) / (confusion_matrix["precision"] + confusion_matrix["recall"])
confusion_matrix["accuracy"] = (confusion_matrix["TP"] + confusion_matrix["TN"]) / (confusion_matrix["n"])
confusion_matrix["specificity"] = confusion_matrix["TN"] / (confusion_matrix["TN"] + confusion_matrix["FP"])

confusion_matrix.round(2)

,FP,TN,FN,TP,n,precision,recall,f1,accuracy,specificity
090,26,0,0,99,125,0.79,1.00,0.88,0.79,0.00
093,9,17,14,85,125,0.90,0.86,0.88,0.82,0.65
095,7,19,18,81,125,0.92,0.82,0.87,0.80,0.73
097,0,26,30,69,125,1.00,0.70,0.82,0.76,1.00


#### Conclusion


This approach is not perfect. There are listings being flagged as duplicates albeit differing slightly in job title or job description. 
E.g. there are listings that have similarity scores of 1.0 but different job titles: Data Scientist/Data Analyst/Data Engineer. 

Some listings that are surely identical show similarity scores of only 0.93, potentially due to "noise" in the description originating from e.g. headlines provided on the different job platforms.

With a threshold of 0.97, the comparison is therefore perhaps too strict. A threshold of 0.90 does however result in some listings beeing incorrectly flagged as duplicates.

In this case, the individual datapoints are pretty valuable. The error of dropping unique job postings that might potentially be _the_ position for an applicant outweighs keeping 2 identical postings that ultimately would only marginally affect the percentage values of job statistics displayed.

Hence, the most conservative threshold of 0.97 will be selected, which most reliably guarantees not dropping unique postings.

#### Aggregate duplicates

In [68]:
# split df into rows without any flags, and those being flagged as duplicates.
print(f"No. rows pre splitting: {len(df)}")
df_duplicates = df[df["similarity_results_0_97_mgroup"].isna() == False].copy()
print(f"No. rows in duplicates df: {len(df_duplicates)}")
df_no_duplicates = df[df["similarity_results_0_97_mgroup"].isna()].copy()
print(f"No. rows in non-duplicates df: {len(df_no_duplicates)}")

No. rows pre splitting: 1468
No. rows in duplicates df: 69
No. rows in non-duplicates df: 1399


In [69]:
df_duplicates.sort_values(by="similarity_results_0_97_mgroup").tail(6)

,title,company,location,salary,seniority_level,date_posted,date_accessed,days_online,number_applicants,job_type_Vollzeit,...,index_pre_agg,index_agg_list,similarity_results_0_90,similarity_results_0_93,similarity_results_0_95,similarity_results_0_97,similarity_results_0_90_mgroup,similarity_results_0_93_mgroup,similarity_results_0_95_mgroup,similarity_results_0_97_mgroup
1014,Senior Scientist Machine Learning – Risikomode...,HUK-COBURG Versicherungsgruppe,Coburg,None,None,None,2024-12-12,NaN,NaN,1,...,1014,[1014],"[(421, 0.903, 94), (571, 0.912, 94), (987, 0.9...","[(987, 0.983, 146), (1050, 0.99, 146)]","[(987, 0.983, 147), (1050, 0.99, 147)]","[(987, 0.983, 148), (1050, 0.99, 148)]",94,146,147,148
1050,Senior Scientist Machine Learning – Causal Mac...,HUK-COBURG Versicherungsgruppe,Coburg,None,None,None,2024-12-12,NaN,NaN,1,...,1050,[1050],"[(421, 0.912, 94), (571, 0.903, 94), (987, 0.9...","[(987, 0.982, 146), (1014, 0.99, 146)]","[(987, 0.982, 147), (1014, 0.99, 147)]","[(987, 0.982, 148), (1014, 0.99, 148)]",94,146,147,148
1596,Data Analytics Consultant (m/w/d),rheindata GmbH,Köln,None,None,2024-12-05,2024-12-13,8.0,NaN,1,...,1596,[1596],"[(1597, 1.0, 167)]","[(1597, 1.0, 168)]","[(1597, 1.0, 169)]","[(1597, 1.0, 170)]",167,168,169,170
1597,Data Analytics Consultant Frontend (m/w/d),rheindata GmbH,Köln,None,None,2024-12-05,2024-12-13,8.0,NaN,1,...,1597,[1597],"[(1596, 1.0, 167)]","[(1596, 1.0, 168)]","[(1596, 1.0, 169)]","[(1596, 1.0, 170)]",167,168,169,170
1611,"Principal, Same Day, Long-term planning",Amazon Europe Core,Luxembourg,None,None,2024-12-12,2024-12-13,1.0,NaN,1,...,1611,[1611],"[(1612, 0.988, 171)]","[(1612, 0.988, 172)]","[(1612, 0.988, 173)]","[(1612, 0.988, 174)]",171,172,173,174
1612,"Principal, Same Day Long-Term Planning",Amazon Europe Core,Luxembourg,None,None,2024-12-12,2024-12-13,1.0,NaN,1,...,1612,[1612],"[(1611, 0.988, 171)]","[(1611, 0.988, 172)]","[(1611, 0.988, 173)]","[(1611, 0.988, 174)]",171,172,173,174


In [70]:
# Keep all original columns. Use similar aggregation functions as before.
group_by_columns = ["similarity_results_0_97_mgroup"]

# set aggregation to max for numeric values (except index), 
# and first for object types (except source, which will be concatenated)
aggregation_functions = {col: "first" if df_duplicates[col].dtype == "O" else "max" for col in df_duplicates.columns}

aggregation_functions.update(index_pre_agg = min)
aggregation_functions.update(index_agg_list = sum)
aggregation_functions.update(source_id = sum)

aggregation_functions

{'title': 'first',
 'company': 'first',
 'location': 'first',
 'salary': 'first',
 'seniority_level': 'first',
 'date_posted': 'first',
 'date_accessed': 'first',
 'days_online': 'max',
 'number_applicants': 'max',
 'job_type_Vollzeit': 'max',
 'job_type_Teilzeit': 'max',
 'job_type_Festanstellung': 'max',
 'job_type_BefristeterVertrag': 'max',
 'job_type_Homeoffice': 'max',
 'job_type_FreieMitarbeit': 'max',
 'job_type_Traineeprogramm': 'max',
 'job_type_Berufsausbildung': 'max',
 'job_type_Praktikum': 'max',
 'job_type_Werkstudent': 'max',
 'job_description_anonymized': 'first',
 'source_id': <function sum(iterable, /, start=0)>,
 'location_clean': 'first',
 'company_clean_ish': 'first',
 'company_clean': 'first',
 'title_clean': 'first',
 'seniority_level_new': 'first',
 'seniority_level_5': 'first',
 'index_pre_agg': <function min>,
 'index_agg_list': <function sum(iterable, /, start=0)>,
 'similarity_results_0_90': 'first',
 'similarity_results_0_93': 'first',
 'similarity_results

In [71]:
df_len_pre = len(df_duplicates)
print("number of rows before aggregation:", df_len_pre)

# Aggregate; Restore original index and order
df_duplicates = df_duplicates.groupby(group_by_columns, as_index=False) \
    .agg(aggregation_functions).reset_index(drop=True)
df_duplicates.index = df_duplicates["index_pre_agg"].to_list()
df_duplicates = df_duplicates.sort_index()

print(f"number of rows after aggregation: {len(df_duplicates)} (dif: {df_len_pre - len(df_duplicates)})")

number of rows before aggregation: 69
number of rows after aggregation: 33 (dif: 36)


In [72]:
df_duplicates.tail()

,title,company,location,salary,seniority_level,date_posted,date_accessed,days_online,number_applicants,job_type_Vollzeit,...,index_pre_agg,index_agg_list,similarity_results_0_90,similarity_results_0_93,similarity_results_0_95,similarity_results_0_97,similarity_results_0_90_mgroup,similarity_results_0_93_mgroup,similarity_results_0_95_mgroup,similarity_results_0_97_mgroup
798,"Applied Scientist, Amazon Nova",Amazon Development Center Germany GmbH,Aachen,None,None,None,2024-12-12,NaN,NaN,1,...,798,"[798, 924]","[(924, 1.0, 131)]","[(924, 1.0, 132)]","[(924, 1.0, 133)]","[(924, 1.0, 134)]",131,132,133,134
957,(Senior) Data Scientist (m/w/d) – Schwerpunkt ...,Ausbildung.de,Bochum,55.000 € – 80.000 € pro Jahr,None,2024-12-11,2024-12-12,2.0,NaN,1,...,957,"[957, 1314, 1171]","[(1171, 0.974, 142)]","[(1171, 0.974, 143)]","[(1171, 0.974, 144)]","[(1171, 0.974, 145)]",142,143,144,145
987,Senior Scientist Machine Learning / Deep Learn...,HUK-COBURG Versicherungsgruppe,Coburg,None,None,None,2024-12-12,NaN,NaN,1,...,987,"[987, 1014, 1050]","[(1014, 0.983, 94), (1050, 0.982, 94)]","[(1014, 0.983, 146), (1050, 0.982, 146)]","[(1014, 0.983, 147), (1050, 0.982, 147)]","[(1014, 0.983, 148), (1050, 0.982, 148)]",94,146,147,148
1596,Data Analytics Consultant (m/w/d),rheindata GmbH,Köln,None,None,2024-12-05,2024-12-13,8.0,NaN,1,...,1596,"[1596, 1597]","[(1597, 1.0, 167)]","[(1597, 1.0, 168)]","[(1597, 1.0, 169)]","[(1597, 1.0, 170)]",167,168,169,170
1611,"Principal, Same Day, Long-term planning",Amazon Europe Core,Luxembourg,None,None,2024-12-12,2024-12-13,1.0,NaN,1,...,1611,"[1611, 1612]","[(1612, 0.988, 171)]","[(1612, 0.988, 172)]","[(1612, 0.988, 173)]","[(1612, 0.988, 174)]",171,172,173,174


In [73]:
# Merge aggregated df_duplicate with df again
print(len(df))
df = pd.concat([df_no_duplicates, df_duplicates])
print(len(df))

1468
1432


In [74]:
df.head()

,title,company,location,salary,seniority_level,date_posted,date_accessed,days_online,number_applicants,job_type_Vollzeit,...,index_pre_agg,index_agg_list,similarity_results_0_90,similarity_results_0_93,similarity_results_0_95,similarity_results_0_97,similarity_results_0_90_mgroup,similarity_results_0_93_mgroup,similarity_results_0_95_mgroup,similarity_results_0_97_mgroup
0,Data Science Graduate,Visa,Berlin,None,Associate,2024-11-22,2024-12-09,17.0,200.0,1,...,0,[0],[],[],[],[],<NA>,<NA>,<NA>,<NA>
1,Junior Data Scientist,Xsolla,Berlin,None,Entry level,2024-11-30,2024-12-09,9.0,200.0,1,...,1,"[1, 1183]",[],[],[],[],<NA>,<NA>,<NA>,<NA>
2,Data Scientist,Global Market Solutions - Germany,Frankfurt am Main,None,Mid-Senior level,2024-11-13,2024-12-09,26.0,200.0,1,...,2,"[2, 19]",[],[],[],[],<NA>,<NA>,<NA>,<NA>
3,Data Scientist Intern,MeloTech,Berlin,None,Entry level,2024-01-31,2024-12-09,313.0,200.0,1,...,3,"[3, 1105]",[],[],[],[],<NA>,<NA>,<NA>,<NA>
4,Data Scientist (f/m/d),vent.io,Frankfurt am Main,None,Associate,2024-11-20,2024-12-09,19.0,200.0,1,...,4,[4],[],[],[],[],<NA>,<NA>,<NA>,<NA>


#### Clean up source column

In [75]:
df["source_id"].value_counts()

source_id
[1]          640
[2]          319
[3]          302
[1, 2]        84
[2, 3]        28
[1, 1]        21
[2, 2]        14
[1, 3]        11
[1, 2, 3]      4
[1, 2, 2]      2
[3, 3]         2
[2, 2, 3]      1
[1, 1, 1]      1
[1, 1, 2]      1
[2, 3, 2]      1
[2, 2, 2]      1
Name: count, dtype: int64

In [76]:
df["source_id"] = df["source_id"].apply(lambda x: ", ".join(map(str, set(x))))

df["source_id"].value_counts()

source_id
1          662
2          334
3          304
1, 2        87
2, 3        30
1, 3        11
1, 2, 3      4
Name: count, dtype: int64

## Job Titles 2 - Categorization

### Use ChatGPT to categorize job titles

In [77]:
# Export pre-cleaned job title column where gender-specifications (e.g. "m/w/d") were removed.
# This reduces redundant characters in the strings and potentially eases the categorization.
df["title_clean"].value_counts()

title_clean
Data Scientist                             85
Senior Data Scientist                      44
Data Analyst                               21
Data Engineer                              21
Machine Learning Engineer                  15
                                           ..
Data Engineer / Analyst                     1
Head of / VP - Data Experience              1
Trainee Analytics Engineer                  1
Business Intelligence Workshop              1
Principal, Same Day, Long-term planning     1
Name: count, Length: 1062, dtype: int64

In [78]:
job_titles = pd.Series(df["title_clean"].unique(), name="job_titles")
#job_titles.to_csv("_ChatGPT/job_titles.csv", header=True, index=False)

#### ChatGPT prompt: 
"I will attach a csv file with around 1000 job titles associated with data focused work in the first column. 
I would like you to categorize these into 10 categories including a category "other", and store these in the second column of the attached csv file."

**Response:**

Understood! Please attach the CSV file, and I will process it for you. I’ll analyze the job titles, assign one of 10 categories to each (including "Other" for ambiguous or uncategorizable titles), and update the second column with the assigned category.
The uploaded file contains a single column labeled **"job_titles"** with job titles related to data-focused work. I’ll proceed by categorizing these titles into 10 categories, including an "Other" category for uncategorizable titles. Here’s how I’ll organize it:

Tentative Categories:
- Data Scientist
- Data Analyst
- Machine Learning Engineer
- Data Engineer
- Business Intelligence (BI)
- Research Scientist
- Statistician
- AI/ML Specialist
- Data Consultant
- Other

Let me categorize them now. ​​

The job titles have been categorized, and a new "Category" column has been added to the DataFrame. For example:
- Junior Data Scientist → Data Scientist
- Applied Scientist, Fintech (Fraud) → Other

I’ll now save the updated CSV file with these categories. ​​

The categorized job titles have been saved. You can download the updated file using the link below:

In [79]:
job_titles_with_categories = pd.read_csv("_ChatGPT/job_titles_with_categories.csv")

In [80]:
# explore provided results
job_titles_with_categories.head()

,job_titles,Category
0,Data Science Graduate,Other
1,Junior Data Scientist,Data Scientist
2,Data Scientist,Data Scientist
3,Data Scientist Intern,Data Scientist
4,"Applied Scientist, Fintech (Fraud)",Other


In [81]:
job_titles_with_categories["Category"].value_counts()

Category
Other                         342
Data Scientist                200
Data Engineer                 145
Data Analyst                  128
AI/ML Specialist               72
Machine Learning Engineer      69
Research Scientist             47
Data Consultant                35
Business Intelligence (BI)     24
Statistician                    1
Name: count, dtype: int64

#### Inspect ChatGPT's analysis

In [82]:
# Load the uploaded CSV file
#job_titles_df = pd.read_csv(file_path)
job_titles_df = pd.DataFrame(job_titles)

# Define a function to categorize job titles
def categorize_job_title(title):
    title_lower = title.lower()
    
    if "data scientist" in title_lower:
        return "Data Scientist"
    elif "analyst" in title_lower:
        return "Data Analyst"
    elif "machine learning" in title_lower or "ml engineer" in title_lower:
        return "Machine Learning Engineer"
    elif "data engineer" in title_lower or "engineer" in title_lower:
        return "Data Engineer"
    elif "business intelligence" in title_lower or "bi " in title_lower:
        return "Business Intelligence (BI)"
    elif "research scientist" in title_lower or "research" in title_lower:
        return "Research Scientist"
    elif "statistician" in title_lower or "statistics" in title_lower:
        return "Statistician"
    elif "ai" in title_lower or "artificial intelligence" in title_lower:
        return "AI/ML Specialist"
    elif "consultant" in title_lower:
        return "Data Consultant"
    else:
        return "Other"

# Apply the categorization function to the job titles
job_titles_df['Category'] = job_titles_df['job_titles'].apply(categorize_job_title)
job_titles_df.head()

,job_titles,Category
0,Data Science Graduate,Other
1,Junior Data Scientist,Data Scientist
2,Data Scientist,Data Scientist
3,Data Scientist Intern,Data Scientist
4,"Applied Scientist, Fintech (Fraud)",Other


#### Apply function to existing dataframe

In [83]:
# Use function to add new column to existing data frame
df["title_short_ChatGPT"] = df["title_clean"].apply(categorize_job_title)

In [84]:
df.filter(like="title").head()

,title,title_clean,title_short_ChatGPT
0,Data Science Graduate,Data Science Graduate,Other
1,Junior Data Scientist,Junior Data Scientist,Data Scientist
2,Data Scientist,Data Scientist,Data Scientist
3,Data Scientist Intern,Data Scientist Intern,Data Scientist
4,Data Scientist (f/m/d),Data Scientist,Data Scientist


In [85]:
df["title_short_ChatGPT"].value_counts()

title_short_ChatGPT
Data Scientist                394
Other                         369
Data Engineer                 205
Data Analyst                  161
Machine Learning Engineer      93
AI/ML Specialist               84
Research Scientist             50
Data Consultant                50
Business Intelligence (BI)     25
Statistician                    1
Name: count, dtype: int64

#### Fine-Tuning 1:

Can you update your code such that e.g. "Data Science Intern" will also be categorized as "Data Scientist"? And can you please refine the condition for "data analyst", since "analyst" alone will also trigger e.g. "Business Analyst".

**Response:**

Got it! I'll refine the code to ensure more accurate categorization. Here's what I'll adjust:

1. "Data Science Intern" and similar titles will be categorized as "Data Scientist."
2. Refined categorization for "Data Analyst":
- Only match titles that explicitly mention "Data Analyst" instead of just "analyst" (to avoid misclassifications like "Business Analyst").

Let me implement these refinements now.

In [86]:
# Refine the categorization function
def refined_categorize_job_title(title):
    title_lower = title.lower()
    
    if "data scientist" in title_lower or "data science" in title_lower:
        return "Data Scientist"
    elif "data analyst" in title_lower:
        return "Data Analyst"
    elif "machine learning" in title_lower or "ml engineer" in title_lower:
        return "Machine Learning Engineer"
    elif "data engineer" in title_lower:
        return "Data Engineer"
    elif "business intelligence" in title_lower or "bi " in title_lower:
        return "Business Intelligence (BI)"
    elif "research scientist" in title_lower or "research" in title_lower:
        return "Research Scientist"
    elif "statistician" in title_lower or "statistics" in title_lower:
        return "Statistician"
    elif "ai" in title_lower or "artificial intelligence" in title_lower:
        return "AI/ML Specialist"
    elif "consultant" in title_lower:
        return "Data Consultant"
    else:
        return "Other"

# Apply the refined categorization function to the job titles
# updated_job_titles_df['Category'] = updated_job_titles_df['job_titles'].apply(refined_categorize_job_title)

# Preview the updated DataFrame with refined categories
# updated_job_titles_df.head()

In [87]:
df["title_short_ChatGPT2"] = df["title_clean"].apply(refined_categorize_job_title)

In [88]:
df.filter(like="title")[(df["title_short_ChatGPT"] != df["title_short_ChatGPT2"])]

,title,title_clean,title_short_ChatGPT,title_short_ChatGPT2
0,Data Science Graduate,Data Science Graduate,Other,Data Scientist
9,AI Engineer,AI Engineer,Data Engineer,AI/ML Specialist
13,Data Science Intern,Data Science Intern,Other,Data Scientist
24,AI&Business Analyst / Consultant (Remote Inter...,AI&Business Analyst / Consultant (Remote Inter...,Data Analyst,AI/ML Specialist
27,AI Engineer (Intern),AI Engineer (Intern),Data Engineer,AI/ML Specialist
...,...,...,...,...
1613,Solutions Engineer,Solutions Engineer,Data Engineer,Other
120,Bachelor of Science (m/w/d) Data Science und K...,Bachelor of Science Data Science und Künstlic...,Other,Data Scientist
213,Research Associate for the Project “DASHH - Da...,Research Associate for the Project “DASHH - Da...,Research Scientist,Data Scientist
250,Senior Software Engineer (Python),Senior Software Engineer (Python),Data Engineer,Other


In [89]:
df.loc[(df["title_short_ChatGPT"] != df["title_short_ChatGPT2"]), "title_short_ChatGPT2"].value_counts()

title_short_ChatGPT2
Other                         113
Data Scientist                106
AI/ML Specialist               46
Business Intelligence (BI)     23
Research Scientist              7
Data Engineer                   1
Data Consultant                 1
Name: count, dtype: int64

In [90]:
df.loc[(df["title_short_ChatGPT"] != df["title_short_ChatGPT2"]), "title_short_ChatGPT"].value_counts()

title_short_ChatGPT
Data Engineer                 130
Data Analyst                   74
Other                          57
AI/ML Specialist               21
Data Consultant                 8
Machine Learning Engineer       2
Research Scientist              2
Business Intelligence (BI)      2
Statistician                    1
Name: count, dtype: int64

In [91]:
# The provided category "AI/ML Specialist" seems a bit inflated and not too accurate.
# -> There do exist many jobs titled "AI Engineer"
df.loc[df["title_short_ChatGPT2"] == "AI/ML Specialist", "title"].sort_values().unique()

array(['(Junior) AI Engineer (m/w/d)',
       '(Junior) Consultant | Gen AI Finance (m/w/d)',
       '(Senior) Consultant AI Specialist (m/w/d)',
       '(Senior) Consultant Tech & AI Implementation (m/w/d)',
       '(Senior) Data & AI PreSales Consultant (m/w/d)',
       'AI & Data - Analyst',
       'AI (Agent) Engineer (f/m/d) | Jobs at VisionAI - Jobs',
       'AI Architect & Optimization Specialist (m/w/d)',
       'AI Cloud Engineer', 'AI Consultant', 'AI Developer',
       'AI Developer (m/w/d)', 'AI Engineer', 'AI Engineer (Intern)',
       'AI Engineer (Senior)', 'AI Engineer (m/w/d)',
       'AI Engineer - Germany (Remote)',
       'AI Engineer / AI Developer (m/w/d) bei einem SaaS-Unternehmen aus Würzburg (Full-Remote oder vor Ort)',
       'AI Engineer for AI Startup',
       'AI Engineer mit Schwerpunkt NLP (m/w/d)', 'AI Lead',
       'AI Platform Engineer (all genders)',
       'AI Software Developer in Test (f/m/d)', 'AI Software Engineer',
       'AI Software Engineer (

In [92]:
# Also, provided category "Statistician" does not appear anymore.
df["title_short_ChatGPT2"].value_counts()

title_short_ChatGPT2
Data Scientist                500
Other                         425
AI/ML Specialist              109
Machine Learning Engineer      91
Data Analyst                   87
Data Engineer                  76
Research Scientist             55
Business Intelligence (BI)     46
Data Consultant                43
Name: count, dtype: int64

#### Fine-Tuning 2:
Can you drop the category "Statistician" and instead include the category "AI Engineer"?


**Response:**

Understood! I will update the code to:

1. Remove the "Statistician" category.
2. Add an "AI Engineer" category to classify relevant titles such as those mentioning "AI Engineer" or similar terms.
Let me make these changes now.

The categorization function has been updated:

1. "Statistician" category removed.
2. Added "AI Engineer" category for relevant titles like "AI Engineer" or "AI" with "Engineer."

The updated categories are now applied to the job titles. I’ll save this refined file for you. ​

In [93]:
# Update the categorization function to remove "Statistician" and add "AI Engineer"
def updated_categorize_job_title(title):
    title_lower = title.lower()
    
    if "data scientist" in title_lower or "data science" in title_lower:
        return "Data Scientist"
    elif "data analyst" in title_lower:
        return "Data Analyst"
    elif "machine learning" in title_lower or "ml engineer" in title_lower:
        return "Machine Learning Engineer"
    elif "data engineer" in title_lower:
        return "Data Engineer"
    elif "business intelligence" in title_lower or "bi " in title_lower:
        return "Business Intelligence (BI)"
    elif "research scientist" in title_lower or "research" in title_lower:
        return "Research Scientist"
    elif "ai engineer" in title_lower or ("ai" in title_lower and "engineer" in title_lower):
        return "AI Engineer"
    elif "ai" in title_lower or "artificial intelligence" in title_lower:
        return "AI/ML Specialist"
    elif "consultant" in title_lower:
        return "Data Consultant"
    else:
        return "Other"

# Apply the updated categorization function to the job titles
# updated_job_titles_df['Category'] = updated_job_titles_df['job_titles'].apply(updated_categorize_job_title)

# # Preview the updated DataFrame
# updated_job_titles_df.head()

In [94]:
df["title_short_ChatGPT3"] = df["title_clean"].apply(updated_categorize_job_title)

In [95]:
df.filter(like="title")[(df["title_short_ChatGPT3"] != df["title_short_ChatGPT2"])]

,title,title_clean,title_short_ChatGPT,title_short_ChatGPT2,title_short_ChatGPT3
9,AI Engineer,AI Engineer,Data Engineer,AI/ML Specialist,AI Engineer
27,AI Engineer (Intern),AI Engineer (Intern),Data Engineer,AI/ML Specialist,AI Engineer
31,ML / AI Engineer Intern (m/f/d),ML / AI Engineer Intern,Data Engineer,AI/ML Specialist,AI Engineer
52,AI Engineer for AI Startup,AI Engineer for AI Startup,Data Engineer,AI/ML Specialist,AI Engineer
72,(Junior) AI Engineer (m/w/d),(Junior) AI Engineer,Data Engineer,AI/ML Specialist,AI Engineer
87,AI Engineer,AI Engineer,Data Engineer,AI/ML Specialist,AI Engineer
105,AI Engineer - Germany (Remote),AI Engineer - Germany (Remote),Data Engineer,AI/ML Specialist,AI Engineer
112,Robotics and Autonomous Systems / Robotics and...,Robotics and Autonomous Systems / Robotics and...,Data Engineer,AI/ML Specialist,AI Engineer
117,AI Engineer,AI Engineer,Data Engineer,AI/ML Specialist,AI Engineer
160,AI Engineer mit Schwerpunkt NLP (m/w/d),AI Engineer mit Schwerpunkt NLP,Data Engineer,AI/ML Specialist,AI Engineer


In [96]:
df["title_short_ChatGPT3"].value_counts()

title_short_ChatGPT3
Data Scientist                500
Other                         425
Machine Learning Engineer      91
Data Analyst                   87
Data Engineer                  76
AI/ML Specialist               71
Research Scientist             55
Business Intelligence (BI)     46
Data Consultant                43
AI Engineer                    38
Name: count, dtype: int64

In [97]:
# See if category "Other" could potentially be further classified (does not seem to be the case).
df.loc[df["title_short_ChatGPT3"] == "Other", "title_clean"].sort_values().unique()

array(['(Advanced) Data Developer', '(Junior) Data Product Associate',
       '(Junior) Digital Produktmanager  AurumCars',
       '(Junior) Digital Produktmanager  Mietwagen',
       '(Junior) Digital Produktmanager  Payment Mietwagen',
       '(Junior) Online-Marketing Produktmanager  Mietwagen',
       '(Junior) Product Manager Financial Services',
       '(Junior) Produktmanager  Chatbot Mietwagen',
       '(Junior) Produktmanager  Operations Analytics Mietwagen',
       '(Junior) Produktmanager  SEO-Automatisierung Mietwagen',
       '(Junior) Produktmanager / Operationsmanager  Automatisierung Mietwagen',
       '(Junior) Specialist ESG Reporting',
       '(Junior) Technischer Produktmanager  Partnermanagement Mietwagen',
       '(Senior) Biochemist / Biopyhsicist',
       '(Senior) Data & Survey Engineer , Quantitative Marktforschung',
       '(Senior) DevOps Data Integration Engineer',
       '(Senior) Scientist Primary Pharmacology of the Target Incubator Unit',
       '(Senio

In [98]:
# Check if category "Data Scientist" is accurate (seems to be the case!)
df.loc[df["title_short_ChatGPT3"] == "Data Scientist", "title_clean"].sort_values().unique()

array(['(Associate) Consultant (Gen) AI & Data Scientist | AI Garage',
       '(Junior) Consultant Data Science & Artificial Intelligence',
       '(Junior) Data Scientist', '(Sen.) Data Scientist',
       '(Senior) Consultant Data Science & MLOp/m',
       '(Senior) Consultant Data Science Time-Series & Forecasting  (Data Scientist)',
       '(Senior) Data Analyst / Data Scientist  in Data Division',
       '(Senior) Data Scientist',
       '(Senior) Data Scientist  Internationales Team',
       '(Senior) Data Scientist  in München',
       '(Senior) Data Scientist  with Spanish language skills',
       '(Senior) Data Scientist  – Schwerpunkt Predictive Modelling und GenAI ab 01.04.25',
       '(Senior) Data Scientist (Analytics), Merchant',
       '(Senior) Data Scientist - Document and Text Analytics',
       '(Senior) Data Scientist / ML Engineer',
       '(Senior) Data Scientist / Researcher',
       '(Senior) Data Scientist Analytics',
       '(Senior) Data Scientist Health Data 

### Conclusion

After refining the initial prompt twice, the resulting categorization seems satisfactory for my use-case. 

Hence, the final function *updated_categorize_job_title* will be used (results stored in df["title_short_ChatGPT3"])

## Number of days online (grouped)

In [99]:
def days_online_grouping(x):
    if x <= 7: return "0-7"
    if x <= 14: return "8-14"
    if x <= 21: return "15-21"
    if x <= 28: return "22-28"
    if x > 28: return "29+"
    if pd.isna(x): return "N/A"

df["days_online_grouped"] = df["days_online"].apply(days_online_grouping)
df["days_online_grouped"].value_counts()

days_online_grouped
0-7      389
N/A      334
8-14     233
29+      204
15-21    177
22-28     95
Name: count, dtype: int64

## Tech Stack

In [100]:
df["job_description_anonymized"].head()

0    Company DescriptionFascinated by the future an...
1    Xsolla is looking for a Junior Data Scientist ...
2    GMS is an operational consultancy with experti...
3    Your missionPropose innovative solutions by ex...
4    The target of vent.io is to accelerate the gro...
Name: job_description_anonymized, dtype: object

In [101]:
# Clean white space characters (e.g. "\n") and non-word characters (i.e. "?/!/:/./...")
df["job_description_for_search"] = df["job_description_anonymized"].replace(r"[\s\W]", " ", regex=True)

In [102]:
df["job_description_for_search"].head()

0    Company DescriptionFascinated by the future an...
1    Xsolla is looking for a Junior Data Scientist ...
2    GMS is an operational consultancy with experti...
3    Your missionPropose innovative solutions by ex...
4    The target of vent io is to accelerate the gro...
Name: job_description_for_search, dtype: object

In [103]:
# Define search terms and create a dummy-variable for each tech stack.
search_terms_clear = [
    "python",
    "julia",
    "spss",
    "stata",
    "javascript",
    "sql|postgre",
    "azure",
    "gcp|google cloud",
    "powerbi|power bi",
    "tableau",
    "qlik",
    "alteryx",
    "docker",
]

# Some search terms are ambigous, i.e. undesired search results may appear.
# These will be wrapped in blanks to ensure that the term matches exact.
search_terms_ambiguous = [
    "rust",             # e.g. "trust"
    "sas",
    "excel",            # e.g. "excellent"
    "aws|amazon web",   # e.g. "laws"
    "git|github",
    "java",             # "javascript"
]

# combine all search terms, specify search mode
search_terms = {search_term: "clear" for search_term in search_terms_clear}
search_terms.update({search_term: "ambigous" for search_term in search_terms_ambiguous})


# Run search / column creation:
for search_term, search_mode in search_terms.items():
    # determine column name; if multiple terms are listed, use first one
    if len(search_term.split("|")) > 0:
        col_name = f"tech_stack_{search_term.split('|')[0]}"
    else:
        col_name = f"tech_stack_{search_term}"
    # determine search mode; if ambigous, wrap searchterm in blanks
    if search_mode == "clear":
        search_string = f"{search_term}"
    else:
        search_string = f" {search_term} "

    df[col_name] = df["job_description_for_search"].str.lower().str.contains(search_string, regex=True).astype(int)

    print(f"{search_string.strip():18}{sum(df[col_name])}")

# R
df["tech_stack_r"] = df["job_description_for_search"].str.contains(" R ").astype(int)
print(f'{"R":18}{sum(df["tech_stack_r"])}')

python            891
julia             19
spss              9
stata             5
javascript        52
sql|postgre       548
azure             236
gcp|google cloud  160
powerbi|power bi  199
tableau           161
qlik              59
alteryx           11
docker            110
rust              11
sas               42
excel             148
aws|amazon web    285
git|github        157
java              146
R                 309


In [104]:
pd.DataFrame({"n": df.filter(like="tech_stack").sum(),
              "%": df.filter(like="tech_stack").mean()})

,n,%
tech_stack_python,891,0.622207
tech_stack_julia,19,0.013268
tech_stack_spss,9,0.006285
tech_stack_stata,5,0.003492
tech_stack_javascript,52,0.036313
tech_stack_sql,548,0.382682
tech_stack_azure,236,0.164804
tech_stack_gcp,160,0.111732
tech_stack_powerbi,199,0.138966
tech_stack_tableau,161,0.112430


## Methods / Concepts (TBD)

In [105]:
# Include search words like "pytorch", "random forest", "deep learning", "ML", "AI", "NLP", ...

## Location 2 - Geodata

### Get long and lat for locations

In [106]:
# REQUIREMENTS (for using Nominatim)

# No heavy uses (an absolute maximum of 1 request per second).
# Provide a valid HTTP Referer or User-Agent identifying the application (stock User-Agents as set by http libraries will not do).
# Clearly display attribution as suitable for your medium.
# Data is provided under the ODbL license which requires to share alike (although small extractions are likely to be covered by fair usage / fair dealing).

# Source: https://operations.osmfoundation.org/policies/nominatim/

To minimize requests, the following lines of code will be executed only once. Results will be stored.

In [107]:
# geolocator = Nominatim(user_agent='job_detail_analysis')

# locations = df["location"].unique()

# location_long_lat_dict = {loc: None for loc in locations}

# for nr, loc in enumerate(locations):
#     if nr % 10 == 0: print(f"Grabbing geodata for location {nr}/{len(locations)}")
#     location = geolocator.geocode(loc)
#     long_lat = tuple([location.latitude, location.longitude])
#     location_long_lat_dict[loc] = long_lat
#     time.sleep(1)

# location_long_lat_dict

In [108]:
# Store geodata as pkl in order to not have to run the geolocator above again

# with open(f'location_long_lat_dict.pkl', 'wb') as f:
#        pickle.dump(location_long_lat_dict, f)

### Load geodata

In [109]:
# open geodata
with open(f'location_long_lat_dict.pkl', 'rb') as f:
    location_long_lat_dict = pickle.load(f)

location_long_lat_dict

{'Berlin': (52.510885, 13.3989367),
 'Frankfurt am Main': (50.1106444, 8.6820917),
 'Bremen': (53.0758196, 8.8071646),
 'Coburg': (50.2582226, 10.964561),
 'Tübingen': (48.5203263, 9.053596),
 'Hamburg': (53.550341, 10.000654),
 'München': (48.1371079, 11.5753822),
 'Deutschland': (51.1638175, 10.4478313),
 'Freiburg': (47.9960901, 7.8494005),
 'Leipzig': (51.3406321, 12.3747329),
 'Heidelberg': (49.4093582, 8.694724),
 'Hessen': (50.6080651, 9.0284647),
 'Dresden': (51.0493286, 13.7381437),
 'Heilbronn': (49.142291, 9.218655),
 'Schwalbach': (50.1490109, 8.5357434),
 'Eschborn': (50.1504737, 8.5607767),
 'Kronberg': (50.1907271, 8.5194693),
 'Hannover': (52.3744779, 9.7385532),
 'Potsdam': (52.4009309, 13.0591397),
 'Nordrhein-Westfalen': (51.4789205, 7.5543751),
 'Düsseldorf': (51.2254018, 6.7763137),
 'Hürth': (50.8807379, 6.876568),
 'Aschheim': (48.1713481, 11.7160347),
 'Köln': (50.938361, 6.959974),
 'Karlsruhe': (49.0068705, 8.4034195),
 'Essen': (51.4582235, 7.0158171),
 'Weil

In [110]:
df["location_lat_long"] = df["location_clean"].map(location_long_lat_dict)

In [111]:
df.filter(like="location")

,location,location_clean,location_lat_long
0,Berlin,Berlin,"(52.510885, 13.3989367)"
1,Berlin,Berlin,"(52.510885, 13.3989367)"
2,Frankfurt am Main,Frankfurt am Main,"(50.1106444, 8.6820917)"
3,Berlin,Berlin,"(52.510885, 13.3989367)"
4,Frankfurt am Main,Frankfurt am Main,"(50.1106444, 8.6820917)"
...,...,...,...
798,Aachen,Aachen,"(50.776351, 6.083862)"
957,Bochum,Bochum,"(51.4818111, 7.2196635)"
987,Coburg,Coburg,"(50.2582226, 10.964561)"
1596,Köln,Köln,"(50.938361, 6.959974)"


In [112]:
# Split columns into lat and long
df["location_lat"] = df["location_lat_long"].map(lambda x: x[0])
df["location_long"] = df["location_lat_long"].map(lambda x: x[1])

In [113]:
df.filter(like="location").sample(5, random_state=42)

,location,location_clean,location_lat_long,location_lat,location_long
133,Deutschland,Deutschland,"(51.1638175, 10.4478313)",51.163818,10.447831
1408,Tettnang,Tettnang,"(47.6716558, 9.5891158)",47.671656,9.589116
498,München,München,"(48.1371079, 11.5753822)",48.137108,11.575382
378,Köln,Köln,"(50.938361, 6.959974)",50.938361,6.959974
1205,Berlin,Berlin,"(52.510885, 13.3989367)",52.510885,13.398937


## Store final dataset

In [114]:
df.columns

Index(['title', 'company', 'location', 'salary', 'seniority_level',
       'date_posted', 'date_accessed', 'days_online', 'number_applicants',
       'job_type_Vollzeit', 'job_type_Teilzeit', 'job_type_Festanstellung',
       'job_type_BefristeterVertrag', 'job_type_Homeoffice',
       'job_type_FreieMitarbeit', 'job_type_Traineeprogramm',
       'job_type_Berufsausbildung', 'job_type_Praktikum',
       'job_type_Werkstudent', 'job_description_anonymized', 'source_id',
       'location_clean', 'company_clean_ish', 'company_clean', 'title_clean',
       'seniority_level_new', 'seniority_level_5', 'index_pre_agg',
       'index_agg_list', 'similarity_results_0_90', 'similarity_results_0_93',
       'similarity_results_0_95', 'similarity_results_0_97',
       'similarity_results_0_90_mgroup', 'similarity_results_0_93_mgroup',
       'similarity_results_0_95_mgroup', 'similarity_results_0_97_mgroup',
       'title_short_ChatGPT', 'title_short_ChatGPT2', 'title_short_ChatGPT3',
       'days

In [115]:
columns_selected = [
    'title', 'title_short_ChatGPT3',
    'company', 'company_clean', 
    'location_clean', 'location_lat', 'location_long',
    'salary', 'seniority_level_new', 'seniority_level_5',
    'days_online', 'days_online_grouped', 'number_applicants', 
    'source_id', 'date_posted', 'date_accessed',
    
    'job_description_anonymized',
    
    'job_type_Vollzeit', 'job_type_Teilzeit', 'job_type_Festanstellung',
    'job_type_BefristeterVertrag', 'job_type_Homeoffice',
    'job_type_FreieMitarbeit', 'job_type_Traineeprogramm',
    'job_type_Berufsausbildung', 'job_type_Praktikum',
    'job_type_Werkstudent',

    'tech_stack_python', 'tech_stack_julia',
    'tech_stack_spss', 'tech_stack_stata', 'tech_stack_javascript',
    'tech_stack_sql', 'tech_stack_azure', 'tech_stack_gcp',
    'tech_stack_powerbi', 'tech_stack_tableau', 'tech_stack_qlik',
    'tech_stack_alteryx', 'tech_stack_docker', 'tech_stack_rust',
    'tech_stack_sas', 'tech_stack_excel', 'tech_stack_aws',
    'tech_stack_git', 'tech_stack_java', 'tech_stack_r', 

    'index_agg_list']

In [116]:
df_out = df.filter(columns_selected).copy()

In [117]:
# check if all selected columns are present in export df
set(columns_selected) - set(df_out.columns)

set()

In [118]:
df_out = df_out.rename(columns={"seniority_level_new": "seniority_level_8",
                                "title_short_ChatGPT3": "title_cat_ChatGPT"})

In [119]:
df_out.head()

,title,title_cat_ChatGPT,company,company_clean,location_clean,location_lat,location_long,salary,seniority_level_8,seniority_level_5,...,tech_stack_alteryx,tech_stack_docker,tech_stack_rust,tech_stack_sas,tech_stack_excel,tech_stack_aws,tech_stack_git,tech_stack_java,tech_stack_r,index_agg_list
0,Data Science Graduate,Data Scientist,Visa,Visa,Berlin,52.510885,13.398937,None,Associate/Mid-Level,Associate/Mid-Level,...,0,0,0,0,0,0,0,0,0,[0]
1,Junior Data Scientist,Data Scientist,Xsolla,Xsolla,Berlin,52.510885,13.398937,None,Junior,Entry level,...,0,0,0,0,0,1,0,0,1,"[1, 1183]"
2,Data Scientist,Data Scientist,Global Market Solutions - Germany,Global Market Solutions - Germany,Frankfurt am Main,50.110644,8.682092,None,Associate/Mid-Level,Associate/Mid-Level,...,0,0,0,0,0,0,0,0,0,"[2, 19]"
3,Data Scientist Intern,Data Scientist,MeloTech,MeloTech,Berlin,52.510885,13.398937,None,Intern,Entry level,...,0,0,0,0,0,0,0,0,0,"[3, 1105]"
4,Data Scientist (f/m/d),Data Scientist,vent.io,vent.io,Frankfurt am Main,50.110644,8.682092,None,Associate/Mid-Level,Associate/Mid-Level,...,0,0,0,0,0,0,0,0,0,[4]


In [120]:
df_out.isna().sum()

title                             0
title_cat_ChatGPT                 0
company                           0
company_clean                     0
location_clean                    0
location_lat                      0
location_long                     0
salary                         1410
seniority_level_8                 0
seniority_level_5                 0
days_online                     334
days_online_grouped               0
number_applicants               668
source_id                         0
date_posted                     334
date_accessed                     0
job_description_anonymized        0
job_type_Vollzeit                 0
job_type_Teilzeit                 0
job_type_Festanstellung           0
job_type_BefristeterVertrag       0
job_type_Homeoffice               0
job_type_FreieMitarbeit           0
job_type_Traineeprogramm          0
job_type_Berufsausbildung         0
job_type_Praktikum                0
job_type_Werkstudent              0
tech_stack_python           

In [121]:
# Store as xlsx and csv
df_out.to_csv("../job_data.csv", index=True, header=True)
df_out.to_excel("job_data.xlsx", index=True, header=True)

## Transform Data into Long-Format for Streamlit Filters

In [122]:
df_out.head()

,title,title_cat_ChatGPT,company,company_clean,location_clean,location_lat,location_long,salary,seniority_level_8,seniority_level_5,...,tech_stack_alteryx,tech_stack_docker,tech_stack_rust,tech_stack_sas,tech_stack_excel,tech_stack_aws,tech_stack_git,tech_stack_java,tech_stack_r,index_agg_list
0,Data Science Graduate,Data Scientist,Visa,Visa,Berlin,52.510885,13.398937,None,Associate/Mid-Level,Associate/Mid-Level,...,0,0,0,0,0,0,0,0,0,[0]
1,Junior Data Scientist,Data Scientist,Xsolla,Xsolla,Berlin,52.510885,13.398937,None,Junior,Entry level,...,0,0,0,0,0,1,0,0,1,"[1, 1183]"
2,Data Scientist,Data Scientist,Global Market Solutions - Germany,Global Market Solutions - Germany,Frankfurt am Main,50.110644,8.682092,None,Associate/Mid-Level,Associate/Mid-Level,...,0,0,0,0,0,0,0,0,0,"[2, 19]"
3,Data Scientist Intern,Data Scientist,MeloTech,MeloTech,Berlin,52.510885,13.398937,None,Intern,Entry level,...,0,0,0,0,0,0,0,0,0,"[3, 1105]"
4,Data Scientist (f/m/d),Data Scientist,vent.io,vent.io,Frankfurt am Main,50.110644,8.682092,None,Associate/Mid-Level,Associate/Mid-Level,...,0,0,0,0,0,0,0,0,0,[4]


### Melt df seperately for job_type and tech_stack, then merge.

#### Job type

In [123]:
# Job Type
cols_for_long_df_job_type = ["location_clean", "title_cat_ChatGPT", "seniority_level_5", "days_online_grouped"]
cols_for_long_df_job_type.extend(df_out.filter(like="job_type_").columns.to_list())

df_for_long_format_job_type = df_out.filter(cols_for_long_df_job_type)
df_for_long_format_job_type["id"] = df_for_long_format_job_type.index

id_vars = ["id", "location_clean", "title_cat_ChatGPT", "seniority_level_5", "days_online_grouped"]
value_vars = df_out.filter(like="job_type_").columns.to_list()

df_for_long_format_job_type = pd.melt(df_for_long_format_job_type, 
                                      id_vars=id_vars, 
                                      value_vars=value_vars, 
                                      var_name="job_type",
                                      value_name="job_type_value").sort_values(by="id")
df_for_long_format_job_type

,id,location_clean,title_cat_ChatGPT,seniority_level_5,days_online_grouped,job_type,job_type_value
0,0,Berlin,Data Scientist,Associate/Mid-Level,15-21,job_type_Vollzeit,1
11456,0,Berlin,Data Scientist,Associate/Mid-Level,15-21,job_type_Praktikum,0
2864,0,Berlin,Data Scientist,Associate/Mid-Level,15-21,job_type_Festanstellung,0
10024,0,Berlin,Data Scientist,Associate/Mid-Level,15-21,job_type_Berufsausbildung,0
12888,0,Berlin,Data Scientist,Associate/Mid-Level,15-21,job_type_Werkstudent,0
...,...,...,...,...,...,...,...
8558,1624,München,Other,Senior,0-7,job_type_FreieMitarbeit,0
9990,1624,München,Other,Senior,0-7,job_type_Traineeprogramm,0
11422,1624,München,Other,Senior,0-7,job_type_Berufsausbildung,0
1398,1624,München,Other,Senior,0-7,job_type_Vollzeit,1


In [124]:
print("#rows:", len(df), "* #vars:", len(value_vars), "should yield:", len(df)*len(value_vars), "rows")
print("#rows in melted df:", len(df_for_long_format_job_type))

print("sum of all value columns =", df.filter(value_vars).sum().sum())
print("#rows in df if only 1s are kept =", sum(df_for_long_format_job_type["job_type_value"] == 1))

#rows: 1432 * #vars: 10 should yield: 14320 rows
#rows in melted df: 14320
sum of all value columns = 2045
#rows in df if only 1s are kept = 2045


In [125]:
df_for_long_format_job_type = df_for_long_format_job_type[df_for_long_format_job_type["job_type_value"] == 1]
df_for_long_format_job_type

,id,location_clean,title_cat_ChatGPT,seniority_level_5,days_online_grouped,job_type,job_type_value
0,0,Berlin,Data Scientist,Associate/Mid-Level,15-21,job_type_Vollzeit,1
1,1,Berlin,Data Scientist,Entry level,8-14,job_type_Vollzeit,1
2,2,Frankfurt am Main,Data Scientist,Associate/Mid-Level,22-28,job_type_Vollzeit,1
11459,3,Berlin,Data Scientist,Entry level,29+,job_type_Praktikum,1
3,3,Berlin,Data Scientist,Entry level,29+,job_type_Vollzeit,1
...,...,...,...,...,...,...,...
4261,1623,München,Other,Senior,8-14,job_type_Festanstellung,1
1397,1623,München,Other,Senior,8-14,job_type_Vollzeit,1
4262,1624,München,Other,Senior,0-7,job_type_Festanstellung,1
7126,1624,München,Other,Senior,0-7,job_type_Homeoffice,1


#### Tech Stack

In [126]:
# Tech Stack
cols_for_long_df_tech_stack = ["location_clean", "title_cat_ChatGPT", "seniority_level_5", "days_online_grouped"]
cols_for_long_df_tech_stack.extend(df_out.filter(like="tech_stack_").columns.to_list())

df_for_long_format_tech_stack = df_out.filter(cols_for_long_df_tech_stack)
df_for_long_format_tech_stack["id"] = df_for_long_format_tech_stack.index

id_vars = ["id", "location_clean", "title_cat_ChatGPT", "seniority_level_5", "days_online_grouped"]
value_vars = df_out.filter(like="tech_stack_").columns.to_list()

df_for_long_format_tech_stack = pd.melt(df_for_long_format_tech_stack, 
                                        id_vars=id_vars, 
                                        value_vars=value_vars, 
                                        var_name="tech_stack",
                                        value_name="tech_stack_value").sort_values(by="id")
df_for_long_format_tech_stack

,id,location_clean,title_cat_ChatGPT,seniority_level_5,days_online_grouped,tech_stack,tech_stack_value
0,0,Berlin,Data Scientist,Associate/Mid-Level,15-21,tech_stack_python,1
25776,0,Berlin,Data Scientist,Associate/Mid-Level,15-21,tech_stack_java,0
17184,0,Berlin,Data Scientist,Associate/Mid-Level,15-21,tech_stack_docker,0
2864,0,Berlin,Data Scientist,Associate/Mid-Level,15-21,tech_stack_spss,0
8592,0,Berlin,Data Scientist,Associate/Mid-Level,15-21,tech_stack_azure,0
...,...,...,...,...,...,...,...
7126,1624,München,Other,Senior,0-7,tech_stack_javascript,0
5694,1624,München,Other,Senior,0-7,tech_stack_stata,0
4262,1624,München,Other,Senior,0-7,tech_stack_spss,0
2830,1624,München,Other,Senior,0-7,tech_stack_julia,0


In [127]:
print("#rows:", len(df), "* #vars:", len(value_vars), "should yield:", len(df)*len(value_vars), "rows")
print("#rows in melted df:", len(df_for_long_format_tech_stack))

print("sum of all value columns =", df.filter(value_vars).sum().sum())
print("#rows in df if only 1s are kept =", sum(df_for_long_format_tech_stack["tech_stack_value"] == 1))

#rows: 1432 * #vars: 20 should yield: 28640 rows
#rows in melted df: 28640
sum of all value columns = 3558
#rows in df if only 1s are kept = 3558


In [128]:
df_for_long_format_tech_stack = df_for_long_format_tech_stack[df_for_long_format_tech_stack["tech_stack_value"] == 1]
df_for_long_format_tech_stack

,id,location_clean,title_cat_ChatGPT,seniority_level_5,days_online_grouped,tech_stack,tech_stack_value
0,0,Berlin,Data Scientist,Associate/Mid-Level,15-21,tech_stack_python,1
7160,0,Berlin,Data Scientist,Associate/Mid-Level,15-21,tech_stack_sql,1
8593,1,Berlin,Data Scientist,Entry level,8-14,tech_stack_azure,1
22913,1,Berlin,Data Scientist,Entry level,8-14,tech_stack_aws,1
7161,1,Berlin,Data Scientist,Entry level,8-14,tech_stack_sql,1
...,...,...,...,...,...,...,...
1392,1618,Großenlüder,Data Engineer,Senior,8-14,tech_stack_python,1
9984,1618,Großenlüder,Data Engineer,Senior,8-14,tech_stack_azure,1
1393,1619,Münchberg,Data Engineer,Senior,8-14,tech_stack_python,1
22875,1621,München,Other,Associate/Mid-Level,0-7,tech_stack_excel,1


#### Merge DFs

In [129]:
df_out_long_merged = df_for_long_format_job_type.combine_first(df_for_long_format_tech_stack)
df_out_long_merged

,days_online_grouped,id,job_type,job_type_value,location_clean,seniority_level_5,tech_stack,tech_stack_value,title_cat_ChatGPT
0,15-21,0,job_type_Vollzeit,1.0,Berlin,Associate/Mid-Level,tech_stack_python,1.0,Data Scientist
1,8-14,1,job_type_Vollzeit,1.0,Berlin,Entry level,tech_stack_python,1.0,Data Scientist
2,22-28,2,job_type_Vollzeit,1.0,Frankfurt am Main,Associate/Mid-Level,tech_stack_python,1.0,Data Scientist
3,29+,3,job_type_Vollzeit,1.0,Berlin,Entry level,tech_stack_python,1.0,Data Scientist
4,15-21,4,job_type_Vollzeit,1.0,Frankfurt am Main,Associate/Mid-Level,tech_stack_python,1.0,Data Scientist
...,...,...,...,...,...,...,...,...,...
28614,8-14,191,NaN,NaN,Berlin,Associate/Mid-Level,tech_stack_r,1.0,Data Analyst
28617,29+,293,NaN,NaN,Hamburg,Entry level,tech_stack_r,1.0,Data Scientist
28629,15-21,549,NaN,NaN,Hamburg,Director,tech_stack_r,1.0,Data Scientist
28632,29+,638,NaN,NaN,Darmstadt,Associate/Mid-Level,tech_stack_r,1.0,AI/ML Specialist


#### Check results

Job Type

In [130]:
df_out_long_merged["job_type"].value_counts()

job_type
job_type_Vollzeit              1162
job_type_Festanstellung         363
job_type_Homeoffice             216
job_type_Teilzeit               151
job_type_Praktikum               75
job_type_Werkstudent             33
job_type_BefristeterVertrag      20
job_type_FreieMitarbeit          19
job_type_Traineeprogramm          3
job_type_Berufsausbildung         3
Name: count, dtype: int64

In [131]:
df.filter(like="job_type").sum()

job_type_Vollzeit              1162
job_type_Teilzeit               151
job_type_Festanstellung         363
job_type_BefristeterVertrag      20
job_type_Homeoffice             216
job_type_FreieMitarbeit          19
job_type_Traineeprogramm          3
job_type_Berufsausbildung         3
job_type_Praktikum               75
job_type_Werkstudent             33
dtype: int64

Tech Stack

In [132]:
df_out_long_merged["tech_stack"].value_counts()

tech_stack
tech_stack_python        891
tech_stack_sql           548
tech_stack_r             309
tech_stack_aws           285
tech_stack_azure         236
tech_stack_powerbi       199
tech_stack_tableau       161
tech_stack_gcp           160
tech_stack_git           157
tech_stack_excel         148
tech_stack_java          146
tech_stack_docker        110
tech_stack_qlik           59
tech_stack_javascript     52
tech_stack_sas            42
tech_stack_julia          19
tech_stack_alteryx        11
tech_stack_rust           11
tech_stack_spss            9
tech_stack_stata           5
Name: count, dtype: int64

In [133]:
df.filter(like="tech_stack").sum()

tech_stack_python        891
tech_stack_julia          19
tech_stack_spss            9
tech_stack_stata           5
tech_stack_javascript     52
tech_stack_sql           548
tech_stack_azure         236
tech_stack_gcp           160
tech_stack_powerbi       199
tech_stack_tableau       161
tech_stack_qlik           59
tech_stack_alteryx        11
tech_stack_docker        110
tech_stack_rust           11
tech_stack_sas            42
tech_stack_excel         148
tech_stack_aws           285
tech_stack_git           157
tech_stack_java          146
tech_stack_r             309
dtype: int64

Job Title

In [134]:
df_out_long_merged.groupby("id").agg({"title_cat_ChatGPT": "first"}).value_counts()

title_cat_ChatGPT         
Data Scientist                495
Other                         412
Machine Learning Engineer      91
Data Analyst                   87
Data Engineer                  76
AI/ML Specialist               71
Research Scientist             53
Business Intelligence (BI)     46
Data Consultant                43
AI Engineer                    38
Name: count, dtype: int64

In [135]:
df_out["title_cat_ChatGPT"].value_counts()

title_cat_ChatGPT
Data Scientist                500
Other                         425
Machine Learning Engineer      91
Data Analyst                   87
Data Engineer                  76
AI/ML Specialist               71
Research Scientist             55
Business Intelligence (BI)     46
Data Consultant                43
AI Engineer                    38
Name: count, dtype: int64

In [136]:
# There n=20 ads without any job type or tech stack match...

In [137]:
df_out_long_merged_ids = set(df_out_long_merged["id"])
df_out_ids = set(df_out.index)

print("#ids in long df:", len(df_out_long_merged_ids))
print("#ids in wide df:", len(df_out_ids))

#ids in long df: 1412
#ids in wide df: 1432


In [138]:
ids_not_in_long = sorted(list(df_out_ids - df_out_long_merged_ids))
ids_not_in_long

[590,
 930,
 997,
 1000,
 1004,
 1020,
 1023,
 1044,
 1055,
 1059,
 1100,
 1124,
 1136,
 1169,
 1179,
 1182,
 1204,
 1211,
 1227,
 1272]

In [139]:
df.filter(like="tech_stack").loc[ids_not_in_long].sum()

tech_stack_python        0
tech_stack_julia         0
tech_stack_spss          0
tech_stack_stata         0
tech_stack_javascript    0
tech_stack_sql           0
tech_stack_azure         0
tech_stack_gcp           0
tech_stack_powerbi       0
tech_stack_tableau       0
tech_stack_qlik          0
tech_stack_alteryx       0
tech_stack_docker        0
tech_stack_rust          0
tech_stack_sas           0
tech_stack_excel         0
tech_stack_aws           0
tech_stack_git           0
tech_stack_java          0
tech_stack_r             0
dtype: int64

In [140]:
df.filter(like="job_type").loc[ids_not_in_long].sum()

job_type_Vollzeit              0
job_type_Teilzeit              0
job_type_Festanstellung        0
job_type_BefristeterVertrag    0
job_type_Homeoffice            0
job_type_FreieMitarbeit        0
job_type_Traineeprogramm       0
job_type_Berufsausbildung      0
job_type_Praktikum             0
job_type_Werkstudent           0
dtype: int64

As expected, those IDs do not show any positive value in any of these columns and were therefore filtered out.

They will be added to the long-format dataframe again.

In [141]:
id_vars = ["location_clean", "title_cat_ChatGPT", "seniority_level_5", "days_online_grouped"]

In [142]:
df_out_long_merged_add_ids = df_out.loc[ids_not_in_long, id_vars]
df_out_long_merged_add_ids["id"] = df_out_long_merged_add_ids.index
df_out_long_merged_add_ids.tail()

,location_clean,title_cat_ChatGPT,seniority_level_5,days_online_grouped,id
1182,Karlsruhe,Other,Associate/Mid-Level,N/A,1182
1204,Göttingen,Other,Postdoc,N/A,1204
1211,Berlin,Other,Associate/Mid-Level,N/A,1211
1227,Berlin,Other,Associate/Mid-Level,N/A,1227
1272,Hamburg,Other,Senior,N/A,1272


In [143]:
df_out_long_merged_all_ids = pd.concat([df_out_long_merged, df_out_long_merged_add_ids])
df_out_long_merged_all_ids.tail()

,days_online_grouped,id,job_type,job_type_value,location_clean,seniority_level_5,tech_stack,tech_stack_value,title_cat_ChatGPT
1182,N/A,1182,NaN,NaN,Karlsruhe,Associate/Mid-Level,NaN,NaN,Other
1204,N/A,1204,NaN,NaN,Göttingen,Postdoc,NaN,NaN,Other
1211,N/A,1211,NaN,NaN,Berlin,Associate/Mid-Level,NaN,NaN,Other
1227,N/A,1227,NaN,NaN,Berlin,Associate/Mid-Level,NaN,NaN,Other
1272,N/A,1272,NaN,NaN,Hamburg,Senior,NaN,NaN,Other


In [144]:
print(len(set(df_out_long_merged_all_ids["id"])))

1432


In [145]:
df_out_long_merged_all_ids.groupby(by="id").agg({"title_cat_ChatGPT": "first"}).value_counts()

title_cat_ChatGPT         
Data Scientist                500
Other                         425
Machine Learning Engineer      91
Data Analyst                   87
Data Engineer                  76
AI/ML Specialist               71
Research Scientist             55
Business Intelligence (BI)     46
Data Consultant                43
AI Engineer                    38
Name: count, dtype: int64

In [146]:
df_out["title_cat_ChatGPT"].value_counts()

title_cat_ChatGPT
Data Scientist                500
Other                         425
Machine Learning Engineer      91
Data Analyst                   87
Data Engineer                  76
AI/ML Specialist               71
Research Scientist             55
Business Intelligence (BI)     46
Data Consultant                43
AI Engineer                    38
Name: count, dtype: int64

...Looks better

### Clean job_type and tech_stack columns

In [147]:
df_out_long_merged_all_ids.head()

,days_online_grouped,id,job_type,job_type_value,location_clean,seniority_level_5,tech_stack,tech_stack_value,title_cat_ChatGPT
0,15-21,0,job_type_Vollzeit,1.0,Berlin,Associate/Mid-Level,tech_stack_python,1.0,Data Scientist
1,8-14,1,job_type_Vollzeit,1.0,Berlin,Entry level,tech_stack_python,1.0,Data Scientist
2,22-28,2,job_type_Vollzeit,1.0,Frankfurt am Main,Associate/Mid-Level,tech_stack_python,1.0,Data Scientist
3,29+,3,job_type_Vollzeit,1.0,Berlin,Entry level,tech_stack_python,1.0,Data Scientist
4,15-21,4,job_type_Vollzeit,1.0,Frankfurt am Main,Associate/Mid-Level,tech_stack_python,1.0,Data Scientist


#### Job Type

In [148]:
df_out_long_merged_all_ids["job_type"] = df_out_long_merged_all_ids["job_type"].str.replace("job_type_", "")

In [149]:
df_out_long_merged_all_ids["job_type"].value_counts(dropna=False)

job_type
NaN                   2810
Vollzeit              1162
Festanstellung         363
Homeoffice             216
Teilzeit               151
Praktikum               75
Werkstudent             33
BefristeterVertrag      20
FreieMitarbeit          19
Traineeprogramm          3
Berufsausbildung         3
Name: count, dtype: int64

In [150]:
job_type_dict = {
    "Vollzeit"           : "Full-time",     
    "Festanstellung"     : "Permanent",    
    "Homeoffice"         : "Homeoffice",        
    "Teilzeit"           : "Part-time",          
    "Praktikum"          : "Internship",         
    "Werkstudent"        : "Working student",       
    "BefristeterVertrag" : "Fixed-term",
    "FreieMitarbeit"     : "Freelance",    
    "Traineeprogramm"    : "Traineeprogram", 
    "Berufsausbildung"   : "Vocational training", 
}

df_out_long_merged_all_ids["job_type"] = df_out_long_merged_all_ids["job_type"].map(job_type_dict)

In [151]:
df_out_long_merged_all_ids.head()

,days_online_grouped,id,job_type,job_type_value,location_clean,seniority_level_5,tech_stack,tech_stack_value,title_cat_ChatGPT
0,15-21,0,Full-time,1.0,Berlin,Associate/Mid-Level,tech_stack_python,1.0,Data Scientist
1,8-14,1,Full-time,1.0,Berlin,Entry level,tech_stack_python,1.0,Data Scientist
2,22-28,2,Full-time,1.0,Frankfurt am Main,Associate/Mid-Level,tech_stack_python,1.0,Data Scientist
3,29+,3,Full-time,1.0,Berlin,Entry level,tech_stack_python,1.0,Data Scientist
4,15-21,4,Full-time,1.0,Frankfurt am Main,Associate/Mid-Level,tech_stack_python,1.0,Data Scientist


#### Tech Stack

In [152]:
df_out_long_merged_all_ids["tech_stack"] = df_out_long_merged_all_ids["tech_stack"].str.replace("tech_stack_", "").str.capitalize()

In [153]:
df_out_long_merged_all_ids["tech_stack"].value_counts()

tech_stack
Python        891
Sql           548
R             309
Aws           285
Azure         236
Powerbi       199
Tableau       161
Gcp           160
Git           157
Excel         148
Java          146
Docker        110
Qlik           59
Javascript     52
Sas            42
Julia          19
Alteryx        11
Rust           11
Spss            9
Stata           5
Name: count, dtype: int64

In [154]:
tech_stack_dict = {
    "Sql"       : "SQL",
    "Aws"       : "AWS",
    "Powerbi"   : "Power BI",
    "Gcp"       : "GCP",
    "Javascript": "JavaScript",
    "Sas"       : "SAS",
    "Spss"      : "SPSS",
}

def clean_tech_stack(x):
    if x in tech_stack_dict:
        return tech_stack_dict[x]
    return x

df_out_long_merged_all_ids["tech_stack"] = df_out_long_merged_all_ids["tech_stack"].apply(clean_tech_stack)
df_out_long_merged_all_ids["tech_stack"].value_counts()

tech_stack
Python        891
SQL           548
R             309
AWS           285
Azure         236
Power BI      199
Tableau       161
GCP           160
Git           157
Excel         148
Java          146
Docker        110
Qlik           59
JavaScript     52
SAS            42
Julia          19
Alteryx        11
Rust           11
SPSS            9
Stata           5
Name: count, dtype: int64

### Store final long-format dataframe

In [155]:
keep_cols = ["id", "location_clean", "title_cat_ChatGPT", "seniority_level_5", "days_online_grouped", "job_type", "tech_stack"]

df_out_long_merged_all_ids_out = df_out_long_merged_all_ids.filter(keep_cols)
df_out_long_merged_all_ids_out = df_out_long_merged_all_ids_out.fillna("-")
df_out_long_merged_all_ids_out.sample(10, random_state=42)

,id,location_clean,title_cat_ChatGPT,seniority_level_5,days_online_grouped,job_type,tech_stack
24428,92,Hamburg,Machine Learning Engineer,Associate/Mid-Level,29+,-,Git
4231,1586,Köln,Machine Learning Engineer,Associate/Mid-Level,0-7,Permanent,-
27245,38,Schwalbach,Data Scientist,Entry level,15-21,-,R
1364,1583,Stuttgart,Other,Associate/Mid-Level,0-7,Full-time,-
4003,1348,Rosenheim,Data Scientist,Entry level,8-14,Permanent,-
24389,46,Hannover,Data Scientist,Associate/Mid-Level,0-7,-,Git
22875,1621,München,Other,Associate/Mid-Level,0-7,-,Excel
1381,1604,Mannheim,Other,Associate/Mid-Level,8-14,Full-time,-
23005,101,Frankfurt am Main,Other,Director,0-7,-,AWS
4093,1445,Rosenheim,Data Engineer,Associate/Mid-Level,15-21,Permanent,-


In [156]:
df_out_long_merged_all_ids_out.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4855 entries, 0 to 1272
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   4855 non-null   int64 
 1   location_clean       4855 non-null   object
 2   title_cat_ChatGPT    4855 non-null   object
 3   seniority_level_5    4855 non-null   object
 4   days_online_grouped  4855 non-null   object
 5   job_type             4855 non-null   object
 6   tech_stack           4855 non-null   object
dtypes: int64(1), object(6)
memory usage: 303.4+ KB


In [157]:
# Save as csv
df_out_long_merged_all_ids_out.to_csv("../job_data_long.csv", header=True, index=False)